In [1]:
%%capture

import ROOT
import os
from dask.distributed import Client
from utils import *
from variables import *
from samplesUL import *
ROOT.RDF.Experimental.Distributed.open_files_locally = False
#logger = ROOT.RDF.Experimental.Distributed.create_logger("DEBUG")

redirector = "xrootd-cms.infn.it"
results_dir = "results_UL_test_75wn_3factor"

maxNfilespersample = 999999999

MT = False

distributed = False
nmaxpartitions = 75*3
sched_port = 22446

In [2]:
if distributed != True and MT == True:
    ROOT.ROOT.EnableImplicitMT()

# Import utils functions

In [3]:
#from dask.distributed import Client
#client = Client()  # set up local cluster on your laptop
#client

In [4]:
text_file = open("postselection_UL.h", "r")
data = text_file.read()
def my_initialization_function():
    ROOT.gInterpreter.Declare('{}'.format(data))
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)

else:
    RDataFrame = ROOT.RDataFrame
    my_initialization_function()

input_line_54:818:1: warning: control may reach end of non-void function [-Wreturn-type]
}
^


# Define analysis steps

In [5]:
def filter_trigger(df):
    df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter", "Good vertex selection")
    #df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger") #RR2017
    #df_trigger = df_goodvertex.Filter("HLT_IsoMu24 || HLT_IsoTkMu24 ||  HLT_Ele27_WPTight_Gsf", "Leptonic trigger") ##UL2016*
    df_trigger = df_goodvertex.Filter("HLT_IsoMu27 ||  HLT_Ele35_WPTight_Gsf", "Leptonic trigger") ##UL2017
    #df_trigger = df_goodvertex.Filter("HLT_IsoMu24 ||  HLT_Ele32_WPTight_Gsf", "Leptonic trigger") ##UL2018

    return df_trigger

def select_jets(df):
    df_atleast2Jets = df.Filter("nJet>=2", "At least two jets")
    df_GoodJets = df_atleast2Jets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
    df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
    df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_invmass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, GoodJets_idx)")
    df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets")
    df_jetsDefinitions = df_2VBSjets.Define("Leadingjet_pt", "GetLeading(Jet_pt, VBSJet_idx)")\
                                    .Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)")\
                                    .Define("Leadingjet_phi", "GetLeading(Jet_phi, VBSJet_idx)")\
                                    .Define("Leadingjet_mass", "GetLeading(Jet_mass, VBSJet_idx)")\
                                    .Define("Leadingjet_DeepFlv_b", "GetLeading(Jet_btagDeepFlavB, VBSJet_idx)")\
                                    .Define("Leadingjet_DeepCSVv2_b", "GetLeading(Jet_btagDeepB, VBSJet_idx)")\
                                    .Define("Leadingjet_CSVv2_b", "GetLeading(Jet_btagCSVV2, VBSJet_idx)")\
                                    .Define("SubLeadingjet_pt", "GetSubLeading(Jet_pt, VBSJet_idx)")\
                                    .Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)")\
                                    .Define("SubLeadingjet_phi", "GetSubLeading(Jet_phi, VBSJet_idx)")\
                                    .Define("SubLeadingjet_mass", "GetSubLeading(Jet_mass, VBSJet_idx)")\
                                    .Define("SubLeadingjet_DeepFlv_b", "GetSubLeading(Jet_btagDeepFlavB, VBSJet_idx)")\
                                    .Define("SubLeadingjet_DeepCSVv2_b", "GetSubLeading(Jet_btagDeepB, VBSJet_idx)")\
                                    .Define("SubLeadingjet_CSVv2_b", "GetSubLeading(Jet_btagCSVV2, VBSJet_idx)")
    return df_jetsDefinitions

def select_lepton(df):
    df_selectElectron = df.Define("Electron_idx", "SelectElectron(Electron_pt, Electron_eta, Electron_phi, Electron_jetRelIso, Electron_mvaFall17V2Iso_WPL, Electron_mvaFall17V2Iso_WP90, Jet_eta, Jet_phi, VBSJet_idx, Year)")
    df_selectMuon = df_selectElectron.Define("Muon_idx", "SelectMuon(Muon_pt, Muon_eta, Muon_phi, Muon_tightId, Muon_looseId, Muon_pfRelIso04_all, Jet_eta, Jet_phi, VBSJet_idx)")
    df_atLeast1Lepton = df_selectMuon.Filter("Electron_idx[1] != -1 || Muon_idx[1] != -1", "At least 1 at-least-loose lepton")
    df_goodLeptonFamily = df_atLeast1Lepton.Define("GoodLeptonFamily", "DetermineGoodLepton(HLT_IsoMu27, HLT_Mu50, HLT_Ele35_WPTight_Gsf, HLT_Ele32_WPTight_Gsf_L1DoubleEG, HLT_Photon200, HLT_PFHT250, HLT_PFHT350, Electron_idx, Electron_pt, Electron_eta, Electron_mvaFall17V2Iso_WPL, Electron_jetRelIso, Muon_idx, Muon_pt, Muon_eta, Muon_pfRelIso04_all, Muon_looseId)")
    df_compatibleLeptons = df_goodLeptonFamily.Filter("GoodLeptonFamily != -1 ", "Filter on leptons")
    df_leptonDefinitions = df_compatibleLeptons.Define("Lepton_pt", "GetLepton(Electron_pt, Electron_idx, Muon_corrected_pt, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_eta", "GetLepton(Electron_eta, Electron_idx, Muon_eta, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_phi", "GetLepton(Electron_phi, Electron_idx, Muon_phi, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_mass", "GetLepton(Electron_mass, Electron_idx, Muon_mass, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_charge", "GetLepton(Electron_charge, Electron_idx, Muon_charge, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_genPartFlav", "GetLepton(Electron_genPartFlav, Electron_idx, Muon_genPartFlav, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_TightRegion", "GetLeptonTightFlag(Electron_idx, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_LnTRegion", "1 - Lepton_TightRegion")\
                                                .Define("Lepton_Zeppenfeld", "Zeppenfeld(Lepton_eta, Leadingjet_eta, SubLeadingjet_eta)")\
                                                .Define("Lepton_pdgId", "GetLepton(Electron_pdgId, Electron_idx, Muon_pdgId, Muon_idx, GoodLeptonFamily)")\
                                                .Define("Lepton_SFFake_vsjet2", "SFFakeRatio_lep_calc_vsjet2(Lepton_pt, Lepton_eta, Lepton_pdgId)")\
                                                .Define("Lepton_SFFake_vsjet4", "SFFakeRatio_lep_calc_vsjet4(Lepton_pt, Lepton_eta, Lepton_pdgId)")\
    
    return df_leptonDefinitions

def select_tau(df):
    df_selectTau = df.Define("Tau_idx", "SelectAndVetoTaus(Tau_pt, Tau_eta, Tau_phi, Tau_idDeepTau2017v2p1VSjet, Tau_idDeepTau2017v2p1VSe, Tau_idDeepTau2017v2p1VSmu, GoodLeptonFamily, Electron_idx, Electron_eta, Electron_phi, Muon_idx, Muon_eta, Muon_phi, Jet_eta, Jet_phi, VBSJet_idx, Year)")
    df_1tau = df_selectTau.Filter("Tau_idx[1] != -1", "Exactly 1 at least loose Tau")
    df_tauDefinitions = df_1tau.Define("SelectedTau_pt", "GetTau(Tau_pt, Tau_idx)")\
                               .Define("SelectedTau_eta", "GetTau(Tau_eta, Tau_idx)")\
                               .Define("SelectedTau_phi", "GetTau(Tau_phi, Tau_idx)")\
                               .Define("SelectedTau_mass", "GetTau(Tau_mass, Tau_idx)")\
                               .Define("SelectedTau_charge", "GetTau(Tau_charge, Tau_idx)")\
                               .Define("SelectedTau_relleadtkpt", "GetTau(Tau_leadTkPtOverTauPt, Tau_idx)")\
                               .Define("SelectedTau_decayMode", "GetTau(Tau_decayMode, Tau_idx)")\
                               .Define("SelectedTau_isolation", "GetTau(Tau_neutralIso, Tau_idx)")\
                               .Define("SelectedTau_DeepTau_WP", "pow(GetTau(Tau_idDeepTau2017v2p1VSjet, Tau_idx)*1000., 2) + GetTau(Tau_idDeepTau2017v2p1VSjet, Tau_idx)*1000. + GetTau(Tau_idDeepTau2017v2p1VSe, Tau_idx)*1000.")\
                               .Define("SelectedTau_DeepTauVsEle_raw", "GetLog2(GetTau(Tau_idDeepTau2017v2p1VSe, Tau_idx) + 1)")\
                               .Define("SelectedTau_DeepTauVsMu_raw", "GetLog2(GetTau(Tau_idDeepTau2017v2p1VSmu, Tau_idx) + 1)")\
                               .Define("SelectedTau_DeepTauVsJet_raw", "GetLog2(GetTau(Tau_idDeepTau2017v2p1VSjet, Tau_idx) + 1)")\
                               .Define("SelectedTau_Zeppenfeld", "Zeppenfeld(SelectedTau_eta, Leadingjet_eta, SubLeadingjet_eta)")\
                               .Define("SelectedTau_TightRegion", "Tau_idx[1]")\
                               .Define("SelectedTau_LnTRegion", "1 - Tau_idx[1]")\
                               .Define("SelectedTau_SFFake_vsjet2", "SFFakeRatio_tau_calc_vsjet2(SelectedTau_pt, SelectedTau_eta)")\
                               .Define("SelectedTau_SFFake_vsjet4", "SFFakeRatio_tau_calc_vsjet4(SelectedTau_pt, SelectedTau_eta)")\
                               .Define("SelectedTau_genPartFlav", "GetTau(Tau_genPartFlav, Tau_idx)")\
                               .Define("SelectedTau_jetIdx", "GetTau(Tau_jetIdx, Tau_idx)")

                                #.Define("SelectedTau_GenMatch", "GetTau(Tau_genPartFlav, Tau_idx)")\
    
    return df_tauDefinitions

def scale_factors(df):
    #df_SF = df.Define("wzero", "1")
    df_SF = df.Define("LepSF", "GetLeptonSF(Electron_effSF, Electron_idx, Muon_effSF, Muon_idx, GoodLeptonFamily, IsMC)")\
              .Define("LepSFUp", "GetLeptonSF(Electron_effSF_errUp, Electron_idx, Muon_effSF_errUp, Muon_idx, GoodLeptonFamily, IsMC)")\
              .Define("LepSFDown", "GetLeptonSF(Electron_effSF_errDown, Electron_idx, Muon_effSF_errDown, Muon_idx, GoodLeptonFamily, IsMC)")\
              .Define("TauSF", "getTauSF(SelectedTau_pt, SelectedTau_eta, SelectedTau_genPartFlav, IsMC, Year)")\
              .Define("Tau_vsjet_Down", "TauSF[0][0]")\
              .Define("Tau_vsjet_SF", "TauSF[0][1]")\
              .Define("Tau_vsjet_Up", "TauSF[0][2]")\
              .Define("Tau_vsele_Down", "TauSF[1][0]")\
              .Define("Tau_vsele_SF", "TauSF[1][1]")\
              .Define("Tau_vsele_Up", "TauSF[1][2]")\
              .Define("Tau_vsmu_Down", "TauSF[2][0]")\
              .Define("Tau_vsmu_SF", "TauSF[2][1]")\
              .Define("Tau_vsmu_Up", "TauSF[2][2]")\
              .Define("btagSFs", "btagcalc(GoodJets_idx, Jet_pt, Jet_eta, Jet_partonFlavour, Jet_btagDeepFlavB, Jet_btagSF_deepjet_M_up, Jet_btagSF_deepjet_M_down, Jet_btagSF_deepjet_M, Jet_btagDeepB, IsMC, Year)")\
              .Define("btagSF", "btagSFs[0]")\
              .Define("btagUp", "btagSFs[1]")\
              .Define("btagDown", "btagSFs[2]")\
              .Define("mistagUp", "btagSFs[3]")\
              .Define("mistagDown", "btagSFs[4]")\

    
    
    return df_SF

def general_definitions(df):
    df_definitions = df.Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)")\
                       .Define("deltaPhi_jj", "deltaPhi(Leadingjet_phi, SubLeadingjet_phi)")\
                       .Define("deltaPhi_taulep", "deltaPhi(SelectedTau_phi, Lepton_phi)")\
                       .Define("deltaPhi_tauj1", "deltaPhi(SelectedTau_phi, Leadingjet_phi)")\
                       .Define("deltaPhi_tauj2", "deltaPhi(Leadingjet_phi, SubLeadingjet_phi)")\
                       .Define("deltaPhi_lepj1", "deltaPhi(Lepton_phi, Leadingjet_phi)")\
                       .Define("deltaPhi_lepj2", "deltaPhi(Lepton_phi, SubLeadingjet_phi)")\
                       .Define("deltaEta_jj", "Leadingjet_eta - SubLeadingjet_eta")\
                       .Define("deltaEta_taulep", "SelectedTau_eta - Lepton_eta")\
                       .Define("deltaEta_tauj1", "SelectedTau_eta - Leadingjet_eta")\
                       .Define("deltaEta_tauj2", "SelectedTau_eta - SubLeadingjet_eta")\
                       .Define("deltaEta_lepj1", "Lepton_eta - Leadingjet_eta")\
                       .Define("deltaEta_lepj2", "Lepton_eta - SubLeadingjet_eta")\
                       .Define("deltaTheta_jj", "deltaTheta(Leadingjet_pt, Leadingjet_eta, Leadingjet_phi, Leadingjet_mass, SubLeadingjet_pt, SubLeadingjet_eta, SubLeadingjet_phi, SubLeadingjet_mass)")\
                       .Define("deltaTheta_taulep", "deltaTheta(SelectedTau_pt,SelectedTau_eta, SelectedTau_phi, SelectedTau_mass, Lepton_pt, Lepton_eta, Lepton_phi, Lepton_mass)")\
                       .Define("deltaTheta_tauj1", "deltaTheta(SelectedTau_pt, SelectedTau_eta,SelectedTau_phi, SelectedTau_mass, Leadingjet_pt, Leadingjet_eta, Leadingjet_phi, Leadingjet_mass)")\
                       .Define("deltaTheta_tauj2", "deltaTheta(SelectedTau_pt, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass, SubLeadingjet_pt, SubLeadingjet_eta, SubLeadingjet_phi, SubLeadingjet_mass)")\
                       .Define("Event_Zeppenfeld", "0.5*(Lepton_Zeppenfeld + SelectedTau_Zeppenfeld)")\
                       .Define("Lepton_Zeppenfeld_over_deltaEta_jj", "Lepton_Zeppenfeld / deltaEta_jj")\
                       .Define("SelectedTau_Zeppenfeld_over_deltaEta_jj", "SelectedTau_Zeppenfeld / deltaEta_jj")\
                       .Define("Event_Zeppenfeld_over_deltaEta_jj", "Event_Zeppenfeld / deltaEta_jj")\
                       .Define("mTlepMET", "sqrt(2*Lepton_pt*PuppiMET_pt*(1-cos(Lepton_phi-PuppiMET_phi)))")\
                       .Define("m_1T", "M1T(Lepton_pt, Lepton_eta, Lepton_phi, Lepton_mass, SelectedTau_pt, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass, PuppiMET_pt, PuppiMET_phi)")\
                       .Define("m_o1", "Mo1(Lepton_pt, Lepton_eta, Lepton_phi, Lepton_mass, SelectedTau_pt, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass, PuppiMET_pt, PuppiMET_phi)")\
                       .Define("event_SFFake_vsjet2", "GetEventSFFake(Lepton_SFFake_vsjet2, SelectedTau_SFFake_vsjet2, Lepton_LnTRegion, SelectedTau_LnTRegion)")\
                       .Define("event_SFFake_vsjet4", "GetEventSFFake(Lepton_SFFake_vsjet4, SelectedTau_SFFake_vsjet4, Lepton_LnTRegion, SelectedTau_LnTRegion)")\
                       .Define("m_jjtaulep","GetInvMassNoIndex4(Leadingjet_pt, Leadingjet_eta, Leadingjet_phi, Leadingjet_mass, SubLeadingjet_pt, SubLeadingjet_eta, SubLeadingjet_phi, SubLeadingjet_mass, SelectedTau_pt, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass, Lepton_pt, Lepton_eta, Lepton_phi, Lepton_mass)")\
                       .Define("m_taulep","GetInvMassNoIndex(SelectedTau_pt, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass, Lepton_pt, Lepton_eta, Lepton_phi, Lepton_mass)")\
                       .Define("mT_lep_MET","sqrt(2*Lepton_pt*PuppiMET_pt*(1. - cos(Lepton_phi-PuppiMET_phi)))")\
                       .Define("taujet_RelPt", "taujet_RelPt(SelectedTau_jetIdx, SelectedTau_pt, Jet_pt)")\
                       .Define("taujet_deltaPhi", "taujet_deltaPhi(SelectedTau_jetIdx, SelectedTau_phi, Jet_phi)")\
                       .Define("nBJet", "CountBJets(Jet_pt, Jet_eta, Jet_btagDeepFlavB)")\
                       .Define("event_RT","(Lepton_pt * SelectedTau_pt) / (Leadingjet_pt * SubLeadingjet_pt)")\
                       .Define("m_jjtau","GetInvMassNoIndex3(Leadingjet_pt, Leadingjet_eta, Leadingjet_phi, Leadingjet_mass, SubLeadingjet_pt, SubLeadingjet_eta, SubLeadingjet_phi, SubLeadingjet_mass, SelectedTau_pt, SelectedTau_eta, SelectedTau_phi, SelectedTau_mass)")\
                       .Define("SMbdt", "SMinference(Leadingjet_DeepFlv_b, SubLeadingjet_DeepFlv_b, Event_Zeppenfeld_over_deltaEta_jj, taujet_RelPt, m_o1, event_RT, taujet_deltaPhi, nJet, mT_lep_MET, m_jjtau, m_1T, nBJet, SubLeadingjet_pt, m_jjtaulep, mjj, Leadingjet_pt)");
                       #.Define("SMbdt", "SMinference(mjj, m_jjtaulep, m_taulep, mT_lep_MET, Leadingjet_pt, SubLeadingjet_pt, SelectedTau_mass, MET_pt)");

    return df_definitions

def book_histos(df, region, h):
    h[region] = {} 
    for item in aggregated_samples.items():
        h[region][item[0]] = {}
    h[region]["Fakes"] = {}
    for item in aggregated_samples.items():
        filter_string = ""
        for s in item[1]:
            #print(s.name)
            #print(sampleDict[s.name])
            if filter_string == "":
                filter_string += "Sample == {}".format(sampleDict[s.name])
            else:
                filter_string += " || Sample == {}".format(sampleDict[s.name])
        #print(filter_string)
        for v in variables[region]:
            df_sample = df.Filter(filter_string)
            if item[0] == "Data":
                h[region]["Data"][v._name] = {}
                h[region]["Fakes"][v._name] = {}
                h[region]['Data'][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0")\
                                                   .Filter("event_SFFake_vsjet2 == 0.", "Both lepton and tau are prompt")\
                                                   .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                   .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name)
                h[region]['Fakes'][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0")\
                                                    .Filter("event_SFFake_vsjet2 > 0.", "At least one among lepton and tau is fake")\
                                                    .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                    .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "event_SFFake_vsjet4")
                h[region]['Data'][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1")\
                                                   .Filter("event_SFFake_vsjet4 == 0.", "Both lepton and tau are prompt")\
                                                   .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                   .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name)
                h[region]['Fakes'][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1")\
                                                    .Filter("event_SFFake_vsjet4 > 0.", "At least one among lepton and tau is fake")\
                                                    .Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter")\
                                                    .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "event_SFFake_vsjet2")
            else:
                h[region][item[0]][v._name] = {}
                h[region][item[0]][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0").Filter("Lepton_TightRegion==1").Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter").Filter("SelectedTau_TightRegion==1").Filter("(Lepton_genPartFlav==1|| Lepton_genPartFlav==15) && SelectedTau_genPartFlav==5", "prompt!").Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "wzero")
                h[region][item[0]][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1").Filter("Lepton_TightRegion==1").Filter("(abs(Leadingjet_eta)>3.2||abs(Leadingjet_eta)<2.5) && (abs(SubLeadingjet_eta)>3.2||abs(SubLeadingjet_eta)<2.5)", "Horn filter").Filter("SelectedTau_TightRegion==1").Filter("(Lepton_genPartFlav==1|| Lepton_genPartFlav==15) && SelectedTau_genPartFlav==5", "prompt!").Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "wzero")

def produce_tau_SF(df):
    df_SF = df.Define("FESs", "getFES(Tau_eta, Tau_decayMode, Tau_genPartFlav, IsMC, Year)")\
               .Define("FESDown", "getFlattenedMatrixColumn(FESs, 3, 0)")\
               .Define("FESSF", "getFlattenedMatrixColumn(FESs, 3, 1)")\
               .Define("FESUp", "getFlattenedMatrixColumn(FESs, 3, 2)")\
               .Define("TESs", "getTES(Tau_pt, Tau_decayMode,Tau_genPartFlav, IsMC, Year)")\
               .Define("TESDown", "getFlattenedMatrixColumn(FESs, 3, 0)")\
               .Define("TESSF", "getFlattenedMatrixColumn(FESs, 3, 0)")\
               .Define("TESUp", "getFlattenedMatrixColumn(FESs, 3, 0)")
    return df_SF
                
def jet_tau_redefines(df):
    df_result = df.Redefine("Jet_pt", "Jet_pt_nom")\
                  .Redefine("Jet_mass", "Jet_mass_nom")\
                  .Redefine("FatJet_pt", "FatJet_pt_nom")\
                  .Redefine("FatJet_pt", "FatJet_pt_nom")\
                  .Redefine("FatJet_mass", "FatJet_mass_nom")\
                  .Redefine("FatJet_msoftdrop", "FatJet_msoftdrop_nom")\
                  .Redefine("Tau_pt", "Tau_pt*FESSF*TESSF")\
                  .Redefine("Tau_mass", "Tau_mass*FESSF*TESSF")
    return df_result

#.Define("w_SF", "PrefireWeight*puWeight*LepSF*Tau_vsjet_SF*Tau_vsele_SF*Tau_vsmu_SF*btagSF")


def energetic_variations(df):
    df_sys = df.Vary(["Jet_pt", "Jet_mass", "FatJet_pt", "FatJet_mass", "FatJet_msoftdrop"], "RVec<RVec<RVec<float>>>{{Jet_pt_jesTotalDown, Jet_pt_jesTotalUp}, {Jet_mass_jesTotalDown, Jet_mass_jesTotalUp}, {FatJet_pt_jesTotalDown, FatJet_pt_jesTotalUp}, {FatJet_mass_jesTotalDown, Jet_mass_jesTotalUp}, {FatJet_msoftdrop_jesTotalDown, FatJet_msoftdrop_jesTotalUp}}", variationTags=["down", "up"], variationName="jesTotal")\
               .Vary(["Jet_pt", "Jet_mass", "FatJet_pt", "FatJet_mass", "FatJet_msoftdrop"], "RVec<RVec<RVec<float>>>{{Jet_pt_jerDown, Jet_pt_jerUp}, {Jet_mass_jerDown, Jet_mass_jerUp}, {FatJet_pt_jerDown, FatJet_pt_jerUp}, {FatJet_mass_jerDown, Jet_mass_jerUp}, {FatJet_msoftdrop_jerDown, FatJet_msoftdrop_jerUp}}", variationTags=["down", "up"], variationName="jer")\
               .Vary("TESSF", "RVec<RVec<float>>{TESDown, TESUp}", variationTags=["down", "up"], variationName="TES")\
               .Vary("FESSF", "RVec<RVec<float>>{FESDown, FESUp}", variationTags=["down", "up"], variationName="FES")
    return df_sys

def SF_variations(df):
    df_sys = df.Vary("PrefireWeight", "RVec<float>{PrefireWeight_Down, PrefireWeight_Up}", variationTags=["down", "up"], variationName="PF")\
               .Vary("puWeight", "RVec<float>{puWeightDown, puWeightUp}", variationTags=["down", "up"], variationName="pu")\
               .Vary("btagSF", "RVec<float>{btagDown, btagUp}", variationTags=["down", "up"], variationName="btag")\
               .Vary("LepSF", "RVec<float>{LepSFDown, LepSFUp}", variationTags=["down", "up"], variationName="lep")\
               .Vary("Tau_vsjet_SF", "RVec<float>{Tau_vsjet_Down, Tau_vsjet_Up}", variationTags=["down", "up"], variationName="tau_vsjet")\
               .Vary("Tau_vsele_SF", "RVec<float>{Tau_vsele_Down, Tau_vsele_Up}", variationTags=["down", "up"], variationName="tau_vsele")\
               .Vary("Tau_vsmu_SF", "RVec<float>{Tau_vsmu_Down, Tau_vsmu_Up}", variationTags=["down", "up"], variationName="tau_vsmu")\
               #.Vary("pdf_totalSF", "ROOT::RVecD{pdf_totalDown, pdf_totalUp}", variationTags=["down", "up"], variationName="pdf_total")\
               #.Vary("QCDScaleSF", "ROOT::RVecD{QCDScaleDown, QCDScaleUp}", variationTags=["down", "up"], variationName="QCDScale")\
               #.Vary("ISRSF", "ROOT::RVecD{ISRDown, ISRUp}", variationTags=["down", "up"], variationName="ISR")\
               #.Vary("FSRSF", "ROOT::RVecD{FSRDown, FSRUp}", variationTags=["down", "up"], variationName="FSR")\
    return df_sys

# Define samples to be analyzed

In [6]:
sampleDict = {'ZZtoLep_UL2016APV': 0,'ZZTo2L2Nu_UL2016APV': 1,'ZZTo4L_UL2016APV': 2,'GluGluToContinToZZTo4e_UL2016APV': 3,'GluGluToContinToZZTo2e2mu_UL2016APV': 4,'GluGluToContinToZZTo2e2tau_UL2016APV': 5,'GluGluToContinToZZTo2mu2nu_UL2016APV': 6,'GluGluToContinToZZTo4mu_UL2016APV': 7,'GluGluToContinToZZTo2mu2tau_UL2016APV': 8,'GluGluToContinToZZTo2tau2nu_UL2016APV': 9,'GluGluToContinToZZTo4tau_UL2016APV': 10,'GluGluToContinToZZTo2e2nu_UL2016APV': 11,'TT_UL2016APV': 12,'TT_SemiLep_UL2016APV': 13,'TT_Had_UL2016APV': 14,'TTTo2L2Nu_UL2016APV': 15,'TT_beff_UL2016APV': 16,'TVX_UL2016APV': 17,'TTGJets_UL2016APV': 18,'TTZToQQ_UL2016APV': 19,'TTZToLLNuNu_UL2016APV': 20,'TTWJetsToQQ_UL2016APV': 21,'TTWJetsToLNu_UL2016APV': 22,'tZq_ll_4f_UL2016APV': 23,'VG_UL2016APV': 24,'ZG_UL2016APV': 25,'WG_UL2016APV': 26,'WrongSign_UL2016APV': 27,'WWto2L2Nu_UL2016APV': 28,'GluGluToWWToENEN_UL2016APV': 29,'GluGluToWWToENMN_UL2016APV': 30,'GluGluToWWToENTN_UL2016APV': 31,'GluGluToWWToMNEN_UL2016APV': 32,'GluGluToWWToMNMN_UL2016APV': 33,'GluGluToWWToMNTN_UL2016APV': 34,'GluGluToWWToTNEN_UL2016APV': 35,'GluGluToWWToTNMN_UL2016APV': 36,'GluGluToWWToTNTN_UL2016APV': 37,'ST_tW_top_UL2016APV': 38,'ST_tW_antitop_UL2016APV': 39,'GluGluHToWWTo2L2Nu_UL2016APV': 40,'GluGluHToWWToLNuQQ_UL2016APV': 41,'GluGluHToZZTo4L_UL2016APV': 42,'GluGluHToTauTau_UL2016APV': 43,'VBFHToWWTo2L2Nu_UL2016APV': 44,'VBFHToTauTau_UL2016APV': 45,'ttHToNonbb_UL2016APV': 46,'VHToNonbb_UL2016APV': 47,'Triboson_UL2016APV': 48,'WWTo2L2Nu_DoubleScattering_UL2016': 49,'WWW_4F_UL2016APV': 50,'WWZ_4F_UL2016APV': 51,'WZZ_UL2016APV': 52,'ZZZ_UL2016APV': 53,'WWG_UL2016APV': 54,'WJets_UL2016APV': 55,'WJetsHT70to100_UL2016APV': 56,'WJetsHT100to200_UL2016APV': 57,'WJetsHT200to400_UL2016APV': 58,'WJetsHT400to600_UL2016APV': 59,'WJetsHT600to800_UL2016APV': 60,'WJetsHT800to1200_UL2016APV': 61,'WJetsHT1200to2500_UL2016APV': 62,'WJetsHT2500toInf_UL2016APV': 63,'WZ_UL2016APV': 64,'DYJetsToLL_UL2016APV': 65,'DYJetsToLL_FxFx_UL2016APV': 65,'DYJetsToLL_M10to50_UL2016APV': 66,'DYJetsToLL_M50_UL2016APV': 67,'DYJetsToLL_M50_FxFx_UL2016APV': 67,'DYJetsToLL_M50_UL2016APV_ext': 67,'WpWpJJ_EWK_UL2016APV': 68,'WpWpJJ_QCD_UL2016APV': 69,'VBS_SSWW_SM_UL2016APV': 70,'VBS_SSWW_LL_SM_UL2016APV': 71,'VBS_SSWW_TL_SM_UL2016APV': 72,'VBS_SSWW_TT_SM_UL2016APV': 73,'VBS_SSWW_cW_UL2016APV': 74,'VBS_SSWW_cW_SM_UL2016APV': 75,'VBS_SSWW_cW_BSM_UL2016APV': 76,'VBS_SSWW_cW_INT_UL2016APV': 77,'VBS_SSWW_cHW_UL2016APV': 78,'VBS_SSWW_cHW_SM_UL2016APV': 79,'VBS_SSWW_cHW_BSM_UL2016APV': 80,'VBS_SSWW_cHW_INT_UL2016APV': 81,'VBS_SSWW_cW_cHW_UL2016APV': 82,'VBS_SSWW_DIM6_UL2016APV': 83,'VBS_SSWW_DIM6_SM_UL2016APV': 84,'ZZtoLep_UL2016': 85,'ZZTo2L2Nu_UL2016': 86,'ZZTo4L_UL2016': 87,'GluGluToContinToZZTo4e_UL2016': 88,'GluGluToContinToZZTo2e2mu_UL2016': 89,'GluGluToContinToZZTo2e2tau_UL2016': 90,'GluGluToContinToZZTo2mu2nu_UL2016': 91,'GluGluToContinToZZTo4mu_UL2016': 92,'GluGluToContinToZZTo2mu2tau_UL2016': 93,'GluGluToContinToZZTo2tau2nu_UL2016': 94,'GluGluToContinToZZTo4tau_UL2016': 95,'GluGluToContinToZZTo2e2nu_UL2016': 96,'TT_UL2016': 97,'TT_SemiLep_UL2016': 98,'TT_Had_UL2016': 99,'TTTo2L2Nu_UL2016': 100,'TT_beff_UL2016': 101,'TVX_UL2016': 102,'TTGJets_UL2016': 103,'TTZToQQ_UL2016': 104,'TTZToLLNuNu_UL2016': 105,'TTWJetsToQQ_UL2016': 106,'TTWJetsToLNu_UL2016': 107,'tZq_ll_4f_UL2016': 108,'VG_UL2016': 109,'ZG_UL2016': 110,'WG_UL2016': 111,'WrongSign_UL2016': 112,'WWto2L2Nu_UL2016': 113,'GluGluToWWToENEN_UL2016': 114,'GluGluToWWToENMN_UL2016': 115,'GluGluToWWToENTN_UL2016': 116,'GluGluToWWToMNEN_UL2016': 117,'GluGluToWWToMNMN_UL2016': 118,'GluGluToWWToMNTN_UL2016': 119,'GluGluToWWToTNEN_UL2016': 120,'GluGluToWWToTNMN_UL2016': 121,'GluGluToWWToTNTN_UL2016': 122,'ST_tW_top_UL2016': 123,'ST_tW_antitop_UL2016': 124,'GluGluHToWWTo2L2Nu_UL2016': 125,'GluGluHToWWToLNuQQ_UL2016': 126,'GluGluHToZZTo4L_UL2016': 127,'GluGluHToTauTau_UL2016': 128,'VBFHToWWTo2L2Nu_UL2016': 129,'VBFHToTauTau_UL2016': 130,'ttHToNonbb_UL2016': 131,'VHToNonbb_UL2016': 132,'Triboson_UL2016': 133,'WWW_4F_UL2016': 134,'WWZ_4F_UL2016': 135,'WZZ_UL2016': 136,'ZZZ_UL2016': 137,'WWG_UL2016': 138,'WJets_UL2016': 139,'WJetsHT70to100_UL2016': 140,'WJetsHT100to200_UL2016': 141,'WJetsHT200to400_UL2016': 142,'WJetsHT400to600_UL2016': 143,'WJetsHT600to800_UL2016': 144,'WJetsHT800to1200_UL2016': 145,'WJetsHT1200to2500_UL2016': 146,'WJetsHT2500toInf_UL2016': 147,'WZ_UL2016': 148,'DYJetsToLL_UL2016': 149,'DYJetsToLL_FxFx_UL2016': 149,'DYJetsToLL_M10to50_UL2016': 150,'DYJetsToLL_M50_UL2016': 151,'DYJetsToLL_M50_FxFx_UL2016': 151,'DYJetsToLL_M50_UL2016_ext': 151,'WpWpJJ_EWK_UL2016': 152,'WpWpJJ_QCD_UL2016': 153,'VBS_SSWW_SM_UL2016': 154,'VBS_SSWW_LL_SM_UL2016': 155,'VBS_SSWW_TL_SM_UL2016': 156,'VBS_SSWW_TT_SM_UL2016': 157,'VBS_SSWW_cW_UL2016': 158,'VBS_SSWW_cW_SM_UL2016': 159,'VBS_SSWW_cW_BSM_UL2016': 160,'VBS_SSWW_cW_INT_UL2016': 161,'VBS_SSWW_cHW_UL2016': 162,'VBS_SSWW_cHW_SM_UL2016': 163,'VBS_SSWW_cHW_BSM_UL2016': 164,'VBS_SSWW_cHW_INT_UL2016': 165,'VBS_SSWW_cW_cHW_UL2016': 166,'VBS_SSWW_DIM6_UL2016': 167,'VBS_SSWW_DIM6_SM_UL2016': 168,'ZZtoLep_UL2017': 169,'ZZTo2L2Nu_UL2017': 170,'ZZTo4L_UL2017': 171,'GluGluToContinToZZTo4e_UL2017': 172,'GluGluToContinToZZTo2e2mu_UL2017': 173,'GluGluToContinToZZTo2e2tau_UL2017': 174,'GluGluToContinToZZTo2mu2nu_UL2017': 175,'GluGluToContinToZZTo4mu_UL2017': 176,'GluGluToContinToZZTo2mu2tau_UL2017': 177,'GluGluToContinToZZTo2tau2nu_UL2017': 178,'GluGluToContinToZZTo4tau_UL2017': 179,'GluGluToContinToZZTo2e2nu_UL2017': 180,'TT_UL2017': 181,'TT_SemiLep_UL2017': 182,'TT_Had_UL2017': 183,'TTTo2L2Nu_UL2017': 184,'TT_beff_UL2017': 185,'TVX_UL2017': 186,'TTGJets_UL2017': 187,'TTZToQQ_UL2017': 188,'TTZToLLNuNu_UL2017': 189,'TTWJetsToQQ_UL2017': 190,'TTWJetsToLNu_UL2017': 191,'tZq_ll_4f_UL2017': 192,'VG_UL2017': 193,'ZG_UL2017': 194,'WG_UL2017': 195,'WrongSign_UL2017': 196,'WWto2L2Nu_UL2017': 197,'GluGluToWWToENEN_UL2017': 198,'GluGluToWWToENMN_UL2017': 199,'GluGluToWWToENTN_UL2017': 200,'GluGluToWWToMNEN_UL2017': 201,'GluGluToWWToMNMN_UL2017': 202,'GluGluToWWToMNTN_UL2017': 203,'GluGluToWWToTNEN_UL2017': 204,'GluGluToWWToTNMN_UL2017': 205,'GluGluToWWToTNTN_UL2017': 206,'ST_tW_top_UL2017': 207,'ST_tW_antitop_UL2017': 208,'GluGluHToWWTo2L2Nu_UL2017': 209,'GluGluHToWWToLNuQQ_UL2017': 210,'GluGluHToZZTo4L_UL2017': 211,'GluGluHToTauTau_UL2017': 212,'VBFHToWWTo2L2Nu_UL2017': 213,'VBFHToTauTau_UL2017': 214,'ttHToNonbb_UL2017': 215,'VHToNonbb_UL2017': 216,'Triboson_UL2017': 217,'WWTo2L2Nu_DoubleScattering_UL2017': 218,'WWW_4F_UL2017': 219,'WWZ_4F_UL2017': 220,'WZZ_UL2017': 221,'ZZZ_UL2017': 222,'WWG_UL2017': 223,'WJets_UL2017': 224,'WJetsHT70to100_UL2017': 225,'WJetsHT100to200_UL2017': 226,'WJetsHT200to400_UL2017': 227,'WJetsHT400to600_UL2017': 228,'WJetsHT600to800_UL2017': 229,'WJetsHT800to1200_UL2017': 230,'WJetsHT1200to2500_UL2017': 231,'WJetsHT2500toInf_UL2017': 232,'WZ_UL2017': 233,'DYJetsToLL_UL2017': 234,'DYJetsToLL_FxFx_UL2017': 234,'DYJetsToLL_M10to50_UL2017': 235,'DYJetsToLL_M50_UL2017': 236,'DYJetsToLL_M50_FxFx_UL2017': 236,'DYJetsToLL_M50_UL2017_ext': 236,'WpWpJJ_EWK_UL2017': 237,'WpWpJJ_QCD_UL2017': 238,'VBS_SSWW_SM_UL2017': 239,'VBS_SSWW_LL_SM_UL2017': 240,'VBS_SSWW_TL_SM_UL2017': 241,'VBS_SSWW_TT_SM_UL2017': 242,'VBS_SSWW_cW_UL2017': 243,'VBS_SSWW_cW_SM_UL2017': 244,'VBS_SSWW_cW_BSM_UL2017': 245,'VBS_SSWW_cW_INT_UL2017': 246,'VBS_SSWW_cHW_UL2017': 247,'VBS_SSWW_cHW_SM_UL2017': 248,'VBS_SSWW_cHW_BSM_UL2017': 249,'VBS_SSWW_cHW_INT_UL2017': 250,'VBS_SSWW_cW_cHW_UL2017': 251,'VBS_SSWW_DIM6_UL2017': 252,'VBS_SSWW_DIM6_SM_UL2017': 253,'ZZtoLep_UL2018': 254,'ZZTo2L2Nu_UL2018': 255,'ZZTo4L_UL2018': 256,'GluGluToContinToZZTo4e_UL2018': 257,'GluGluToContinToZZTo2e2mu_UL2018': 258,'GluGluToContinToZZTo2e2tau_UL2018': 259,'GluGluToContinToZZTo2mu2nu_UL2018': 260,'GluGluToContinToZZTo4mu_UL2018': 261,'GluGluToContinToZZTo2mu2tau_UL2018': 262,'GluGluToContinToZZTo2tau2nu_UL2018': 263,'GluGluToContinToZZTo4tau_UL2018': 264,'GluGluToContinToZZTo2e2nu_UL2018': 265,'TT_UL2018': 266,'TT_SemiLep_UL2018': 267,'TT_Had_UL2018': 268,'TTTo2L2Nu_UL2018': 269,'TT_beff_UL2018': 270,'TVX_UL2018': 271,'TTGJets_UL2018': 272,'TTZToQQ_UL2018': 273,'TTZToLLNuNu_UL2018': 274,'TTWJetsToQQ_UL2018': 275,'TTWJetsToLNu_UL2018': 276,'tZq_ll_4f_UL2018': 277,'VG_UL2018': 278,'ZG_UL2018': 279,'WG_UL2018': 280,'WrongSign_UL2018': 281,'WWto2L2Nu_UL2018': 282,'GluGluToWWToENEN_UL2018': 283,'GluGluToWWToENMN_UL2018': 284,'GluGluToWWToENTN_UL2018': 285,'GluGluToWWToMNEN_UL2018': 286,'GluGluToWWToMNMN_UL2018': 287,'GluGluToWWToMNTN_UL2018': 288,'GluGluToWWToTNEN_UL2018': 289,'GluGluToWWToTNMN_UL2018': 290,'GluGluToWWToTNTN_UL2018': 291,'ST_tW_top_UL2018': 292,'ST_tW_antitop_UL2018': 293,'GluGluHToWWTo2L2Nu_UL2018': 294,'GluGluHToWWToLNuQQ_UL2018': 295,'GluGluHToZZTo4L_UL2018': 296,'GluGluHToTauTau_UL2018': 297,'VBFHToWWTo2L2Nu_UL2018': 298,'VBFHToTauTau_UL2018': 299,'ttHToNonbb_UL2018': 300,'VHToNonbb_UL2018': 301,'Triboson_UL2018': 302,'WWTo2L2Nu_DoubleScattering_UL2018': 303,'WWW_4F_UL2018': 304,'WWZ_4F_UL2018': 305,'WZZ_UL2018': 306,'ZZZ_UL2018': 307,'WWG_UL2018': 308,'WJets_UL2018': 309,'WJetsHT70to100_UL2018': 310,'WJetsHT100to200_UL2018': 311,'WJetsHT200to400_UL2018': 312,'WJetsHT400to600_UL2018': 313,'WJetsHT600to800_UL2018': 314,'WJetsHT800to1200_UL2018': 315,'WJetsHT1200to2500_UL2018': 316,'WJetsHT2500toInf_UL2018': 317,'WZ_UL2018': 318,'DYJetsToLL_UL2018': 319,'DYJetsToLL_FxFx_UL2018': 319,'DYJetsToLL_M10to50_UL2018': 320,'DYJetsToLL_M50_UL2018': 321,'DYJetsToLL_M50_FxFx_UL2018': 321,'DYJetsToLL_M50_UL2018_ext': 321,'WpWpJJ_EWK_UL2018': 322,'WpWpJJ_QCD_UL2018': 323,'VBS_SSWW_SM_UL2018': 324,'VBS_SSWW_LL_SM_UL2018': 325,'VBS_SSWW_TL_SM_UL2018': 326,'VBS_SSWW_TT_SM_UL2018': 327,'VBS_SSWW_cW_UL2018': 328,'VBS_SSWW_cW_BSM_UL2018': 329,'VBS_SSWW_cW_SM_UL2018': 330,'VBS_SSWW_cW_INT_UL2018': 331,'VBS_SSWW_cHW_UL2018': 332,'VBS_SSWW_cHW_SM_UL2018': 333,'VBS_SSWW_cHW_BSM_UL2018': 334,'VBS_SSWW_cHW_INT_UL2018': 335,'VBS_SSWW_cW_cHW_UL2018': 336,'VBS_SSWW_DIM6_UL2018': 337,'VBS_SSWW_DIM6_SM_UL2018': 338,'DataMu_UL2016APV': 339,'DataMuB1_UL2016APV': 340,'DataMuB2_UL2016APV': 341,'DataMuC_UL2016APV': 342,'DataMuD_UL2016APV': 343,'DataMuE_UL2016APV': 344,'DataMuF_UL2016APV': 345,'DataMu_UL2016': 346,'DataMuF_UL2016': 347,'DataMuG_UL2016': 348,'DataMuH_UL2016': 349,'DataMu_UL2017': 350,'DataMuB_UL2017': 351,'DataMuC_UL2017': 352,'DataMuD_UL2017': 353,'DataMuE_UL2017': 354,'DataMuF_UL2017': 355,'DataMu_UL2018': 356,'DataMuA_UL2018': 357,'DataMuB_UL2018': 358,'DataMuC_UL2018': 359,'DataMuD_UL2018': 360,'DataEle_UL2016APV': 361,'DataEleB1_UL2016APV': 362,'DataEleB2_UL2016APV': 363,'DataEleC_UL2016APV': 364,'DataEleD_UL2016APV': 365,'DataEleE_UL2016APV': 366,'DataEleF_UL2016APV': 367,'DataEle_UL2016': 368,'DataEleF_UL2016': 369,'DataEleG_UL2016': 370,'DataEleH_UL2016': 371,'DataEle_UL2017': 372,'DataEleB_UL2017': 373,'DataEleC_UL2017': 374,'DataEleD_UL2017': 375,'DataEleE_UL2017': 376,'DataEleF_UL2017': 377,'DataEle_UL2018': 378,'DataEleA_UL2018': 379,'DataEleB_UL2018': 380,'DataEleC_UL2018': 381,'DataEleD_UL2018': 382,'DataHT_UL2016APV': 383,'DataHTB1_UL2016APV': 384,'DataHTB2_UL2016APV': 385,'DataHTC_UL2016APV': 386,'DataHTD_UL2016APV': 387,'DataHTE_UL2016APV': 388,'DataHTF_UL2016APV': 389,'DataHT_UL2016': 390,'DataHTF_UL2016': 391,'DataHTG_UL2016': 392,'DataHTH_UL2016': 393,'DataHT_UL2017': 394,'DataHTB_UL2017': 395,'DataHTC_UL2017': 396,'DataHTD_UL2017': 397,'DataHTE_UL2017': 398,'DataHTF_UL2017': 399,'DataHT_UL2018': 400,'DataHTA_UL2018': 401,'DataHTB_UL2018': 402,'DataHTC_UL2018': 403,'DataHTD_UL2018': 404,'SampleHTFake_UL2016APV': 405,'SampleHTFake_UL2016': 406,'SampleHTFake_UL2017': 407,'SampleHTFake_UL2018': 408,}

aggregated_samples_UL2016APV = {
    'Vgamma':  [ZG_UL2016APV, WG_UL2016APV],
    #'QCD_ssWW': [WpWpJJ_QCD_UL2016APV],   
    'tVX': [TTGJets_UL2016APV, TTZToQQ_UL2016APV, TTZToLLNuNu_UL2016APV, TTWJetsToQQ_UL2016APV, TTWJetsToLNu_UL2016APV, tZq_ll_4f_UL2016APV], 
    'Other': [WWto2L2Nu_UL2016APV, WWW_4F_UL2016APV, WWZ_4F_UL2016APV, WZZ_UL2016APV, ZZZ_UL2016APV, WWG_UL2016APV], 
    'ttbar DiLep': [TTTo2L2Nu_UL2016APV], 
    'WZ': [WZ_UL2016APV], 
    'DY_jets': [DYJetsToLL_M50_FxFx_UL2016APV],
    'Wrong_Sign': [WWto2L2Nu_UL2016APV, GluGluToWWToENEN_UL2016APV, GluGluToWWToENMN_UL2016APV, GluGluToWWToENTN_UL2016APV, GluGluToWWToMNEN_UL2016APV, GluGluToWWToMNMN_UL2016APV, GluGluToWWToMNTN_UL2016APV, GluGluToWWToTNEN_UL2016APV, GluGluToWWToTNMN_UL2016APV, GluGluToWWToTNTN_UL2016APV, ST_tW_top_UL2016APV, GluGluHToWWTo2L2Nu_UL2016APV, GluGluHToZZTo4L_UL2016APV,GluGluHToTauTau_UL2016APV, VBFHToWWTo2L2Nu_UL2016APV, VBFHToTauTau_UL2016APV, ttHToNonbb_UL2016APV, VHToNonbb_UL2016APV], 
    'ZZ': [ZZTo2L2Nu_UL2016APV, ZZTo4L_UL2016APV, GluGluToContinToZZTo2e2nu_UL2016APV, GluGluToContinToZZTo2e2mu_UL2016APV, GluGluToContinToZZTo2e2tau_UL2016APV, GluGluToContinToZZTo2mu2nu_UL2016APV, GluGluToContinToZZTo2mu2tau_UL2016APV, GluGluToContinToZZTo4e_UL2016APV, GluGluToContinToZZTo4mu_UL2016APV, GluGluToContinToZZTo4tau_UL2016APV], 
    'VBS_ssWW': [VBS_SSWW_LL_SM_UL2016APV, VBS_SSWW_TL_SM_UL2016APV, VBS_SSWW_TT_SM_UL2016APV], 
    'Data': [DataEleB2_UL2016APV, DataEleC_UL2016APV, DataEleD_UL2016APV, DataEleE_UL2016APV, DataEleF_UL2016APV, DataMuB2_UL2016APV, DataMuC_UL2016APV, DataMuD_UL2016APV, DataMuE_UL2016APV, DataMuF_UL2016APV], 
}

aggregated_samples_UL2016 = {
    'Vgamma':  [ZG_UL2016, WG_UL2016],
    #'QCD_ssWW': [WpWpJJ_QCD_UL2016],   
    'tVX': [TTGJets_UL2016, TTZToQQ_UL2016, TTZToLLNuNu_UL2016, TTWJetsToQQ_UL2016, TTWJetsToLNu_UL2016, tZq_ll_4f_UL2016], 
    'Other': [WWto2L2Nu_UL2016, WWW_4F_UL2016, WWZ_4F_UL2016, WZZ_UL2016, ZZZ_UL2016, WWG_UL2016], 
    'ttbar DiLep': [TTTo2L2Nu_UL2016], 
    'WZ': [WZ_UL2016], 
    'DY_jets': [DYJetsToLL_M50_FxFx_UL2016],
    'Wrong_Sign': [WWto2L2Nu_UL2016, GluGluToWWToENEN_UL2016, GluGluToWWToENMN_UL2016, GluGluToWWToENTN_UL2016, GluGluToWWToMNEN_UL2016, GluGluToWWToMNMN_UL2016, GluGluToWWToMNTN_UL2016, GluGluToWWToTNEN_UL2016, GluGluToWWToTNMN_UL2016, GluGluToWWToTNTN_UL2016, ST_tW_top_UL2016, GluGluHToWWTo2L2Nu_UL2016, GluGluHToZZTo4L_UL2016,GluGluHToTauTau_UL2016, VBFHToWWTo2L2Nu_UL2016, VBFHToTauTau_UL2016, ttHToNonbb_UL2016, VHToNonbb_UL2016], 
    'ZZ': [ZZTo2L2Nu_UL2016, ZZTo4L_UL2016, GluGluToContinToZZTo2e2nu_UL2016, GluGluToContinToZZTo2e2mu_UL2016, GluGluToContinToZZTo2e2tau_UL2016, GluGluToContinToZZTo2mu2nu_UL2016, GluGluToContinToZZTo2mu2tau_UL2016, GluGluToContinToZZTo4e_UL2016, GluGluToContinToZZTo4mu_UL2016, GluGluToContinToZZTo4tau_UL2016], 
    'VBS_ssWW': [VBS_SSWW_LL_SM_UL2016, VBS_SSWW_TL_SM_UL2016, VBS_SSWW_TT_SM_UL2016], 
    'Data': [DataEleF_UL2016, DataEleG_UL2016, DataEleH_UL2016, DataMuF_UL2016, DataMuG_UL2016, DataMuH_UL2016,], 
}

aggregated_samples_UL2017 = {
    'Vgamma':  [ZG_UL2017, WG_UL2017],
    #'QCD_ssWW': [WpWpJJ_QCD_UL2017],   
    'tVX': [TTGJets_UL2017, TTZToQQ_UL2017, TTZToLLNuNu_UL2017, TTWJetsToQQ_UL2017, TTWJetsToLNu_UL2017, tZq_ll_4f_UL2017], 
    'Other': [WWto2L2Nu_UL2017, WWW_4F_UL2017, WWZ_4F_UL2017, WZZ_UL2017, ZZZ_UL2017, WWG_UL2017], 
    'ttbar DiLep': [TTTo2L2Nu_UL2017], 
    'WZ': [WZ_UL2017], 
    'DY_jets': [DYJetsToLL_M50_FxFx_UL2017],
    'Wrong_Sign': [WWto2L2Nu_UL2017, GluGluToWWToENEN_UL2017, GluGluToWWToENMN_UL2017, GluGluToWWToENTN_UL2017, GluGluToWWToMNEN_UL2017, GluGluToWWToMNMN_UL2017, GluGluToWWToMNTN_UL2017, GluGluToWWToTNEN_UL2017, GluGluToWWToTNMN_UL2017, GluGluToWWToTNTN_UL2017, ST_tW_top_UL2017, GluGluHToWWTo2L2Nu_UL2017, GluGluHToZZTo4L_UL2017,GluGluHToTauTau_UL2017, VBFHToWWTo2L2Nu_UL2017, VBFHToTauTau_UL2017, ttHToNonbb_UL2017, VHToNonbb_UL2017], 
    'ZZ': [ZZTo2L2Nu_UL2017, ZZTo4L_UL2017, GluGluToContinToZZTo2e2nu_UL2017, GluGluToContinToZZTo2e2mu_UL2017, GluGluToContinToZZTo2e2tau_UL2017, GluGluToContinToZZTo2mu2nu_UL2017, GluGluToContinToZZTo2mu2tau_UL2017, GluGluToContinToZZTo4e_UL2017, GluGluToContinToZZTo4mu_UL2017, GluGluToContinToZZTo4tau_UL2017], 
    'VBS_ssWW': [VBS_SSWW_LL_SM_UL2017, VBS_SSWW_TL_SM_UL2017, VBS_SSWW_TT_SM_UL2017], 
    'Data': [DataEleB_UL2017, DataEleC_UL2017, DataEleD_UL2017, DataEleE_UL2017, DataEleF_UL2017, DataMuB_UL2017, DataMuC_UL2017, DataMuD_UL2017, DataMuE_UL2017, DataMuF_UL2017], 
}

aggregated_samples_UL2018 = {
    'Vgamma':  [ZG_UL2018, WG_UL2018],
    #'QCD_ssWW': [WpWpJJ_QCD_UL2018],   
    'tVX': [TTGJets_UL2018, TTZToQQ_UL2018, TTZToLLNuNu_UL2018, TTWJetsToQQ_UL2018, TTWJetsToLNu_UL2018, tZq_ll_4f_UL2018], 
    'Other': [WWto2L2Nu_UL2018, WWW_4F_UL2018, WWZ_4F_UL2018, WZZ_UL2018, ZZZ_UL2018, WWG_UL2018], 
    'ttbar DiLep': [TTTo2L2Nu_UL2018], 
    'WZ': [WZ_UL2018], 
    'DY_jets': [DYJetsToLL_M50_FxFx_UL2018],
    'Wrong_Sign': [WWto2L2Nu_UL2018, GluGluToWWToENEN_UL2018, GluGluToWWToENMN_UL2018, GluGluToWWToENTN_UL2018, GluGluToWWToMNEN_UL2018, GluGluToWWToMNMN_UL2018, GluGluToWWToMNTN_UL2018, GluGluToWWToTNEN_UL2018, GluGluToWWToTNMN_UL2018, GluGluToWWToTNTN_UL2018, ST_tW_top_UL2018, GluGluHToWWTo2L2Nu_UL2018, GluGluHToZZTo4L_UL2018,GluGluHToTauTau_UL2018, VBFHToWWTo2L2Nu_UL2018, VBFHToTauTau_UL2018, ttHToNonbb_UL2018, VHToNonbb_UL2018], 
    'ZZ': [ZZTo2L2Nu_UL2018, ZZTo4L_UL2018, GluGluToContinToZZTo2e2nu_UL2018, GluGluToContinToZZTo2e2mu_UL2018, GluGluToContinToZZTo2e2tau_UL2018, GluGluToContinToZZTo2mu2nu_UL2018, GluGluToContinToZZTo2mu2tau_UL2018, GluGluToContinToZZTo4e_UL2018, GluGluToContinToZZTo4mu_UL2018, GluGluToContinToZZTo4tau_UL2018], 
    'VBS_ssWW': [VBS_SSWW_LL_SM_UL2018, VBS_SSWW_TL_SM_UL2018, VBS_SSWW_TT_SM_UL2018], 
    'Data': [DataEleA_UL2018, DataEleB_UL2018, DataEleC_UL2018, DataEleD_UL2018, DataMuA_UL2018, DataMuB_UL2018, DataMuC_UL2018, DataMuD_UL2018], 
}

aggregated_samples = aggregated_samples_UL2017

In [7]:
#for k,v in sampleDict.items():
#    if "DataMu" in k:
#        print(v,end=", ")

In [8]:
chain = []
proxy = redirector
max_lines = maxNfilespersample

for v in aggregated_samples.values():
    for s in v:
        with open("macros/files/{}.txt".format(s.name)) as file:
            i = 0
            for line in file:
                if i < max_lines: 
                    line = line.rstrip()
                    splitted_parts = line.split("/")
                    splitted_parts[2] = proxy
                    line = "/".join(splitted_parts)
                    chain.append(line)
                i += 1
#chain

In [9]:
chain[0]

'root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/ZG_UL2017/220227_181212/0000/tree_hadd_1.root'

In [10]:
#!xrdcp root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/ZG_UL2017/220227_181212/0000/tree_hadd_1.root .

In [11]:
'''
import ROOT
#%load_ext autoreload
#%autoreload 2
#h
for i in chain:
    if 'DataMuE' in i:
        #print(i)
        try:
            ROOT.TFile.Open(i)
        except:
            print(i)
'''

"\nimport ROOT\n#%load_ext autoreload\n#%autoreload 2\n#h\nfor i in chain:\n    if 'DataMuE' in i:\n        #print(i)\n        try:\n            ROOT.TFile.Open(i)\n        except:\n            print(i)\n"

In [12]:
#!xrdcp root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_UL2017/220227_190016/0000/tree_hadd_1.root VBS_SSWW_LL_SM_UL2017_1.root
#!xrdcp root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/SingleElectron/DataEleC_UL2017/220228_083945/0000/tree_hadd_1.root DataEleC_UL2017_1.root

# Do analysis

In [13]:
h = {}

if distributed == True:
    df = RDataFrame("Events", chain[0], npartitions=nmaxpartitions, daskclient=client)
else:
    chain = ["tree_hadd_1.root"] #root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/ZG_UL2017/220227_181212/0000/tree_hadd_1.root
    df = RDataFrame("Events", chain[0]).Range(1000000)    

df_type = df.Define("IsMC", "isMC(Sample)").Define("Year", "\"UL2017\"")
df_tau_SF = produce_tau_SF(df_type)
df_sys = energetic_variations(df_tau_SF)
df_jet_tau_redefines = jet_tau_redefines(df_sys)

#trigger and VBS jets selections  and e/mu + tau final state
df_trigger = filter_trigger(df_jet_tau_redefines)
df_jets = select_jets(df_trigger)
df_lepton = select_lepton(df_jets).Define("PassDataLeptonCheck", "DataLeptonCheck(Sample, GoodLeptonFamily, IsMC)").Filter("PassDataLeptonCheck == true", "DataLeptonCheck")
df_tau = select_tau(df_lepton)
#df_corrections = corrections(df_tau)
df_definitions = general_definitions(df_tau)
df_SF_def= scale_factors(df_definitions)
df_SF = SF_variations(df_SF_def).Define("w_SF", "PrefireWeight*puWeight*LepSF*Tau_vsjet_SF*Tau_vsele_SF*Tau_vsmu_SF*btagSF")\
                                .Define("Nevents", "getNevents(Sample, IsMC)")\
                                .Define("lumi", "getLumi(Year, IsMC)")\
                                .Define("xsec", "getXSec(Sample, IsMC)")\
                                .Define("w_nominal", "lumi*xsec* 1000./Nevents")\
                                .Define("wzero", "w_SF*w_nominal")

df_SS = df_SF.Filter("SameCharge(GoodLeptonFamily, Electron_idx, Electron_charge, Muon_idx, Muon_charge, Tau_idx, Tau_charge)", "SameSign")
df_OS = df_SF.Filter("SameCharge(GoodLeptonFamily, Electron_idx, Electron_charge, Muon_idx, Muon_charge, Tau_idx, Tau_charge) == false", "OppositeSign")

df_SS_METCut_true = df_SS.Filter("MET_T1_pt > 50", "MET cut true")
df_SS_METCut_false = df_SS.Filter("MET_T1_pt <= 50", "MET cut false")

df_SS_METCut_true_bjets_true = df_SS_METCut_true.Filter("nBJet > 0", "at least one b jets - medium")
df_SS_METCut_true_bjets_false = df_SS_METCut_true.Filter("nBJet == 0", "no b jets - medium")

##### CR fakes
df_FAKES = df_SS_METCut_false
book_histos(df_FAKES, 'CR_fakes', h)
##### SR 
df_SR = df_SS_METCut_true_bjets_false.Filter("mjj > 500", "mjj cut")
book_histos(df_SR, 'SR', h)
#### CR ttbar
df_TTBAR = df_SS_METCut_true_bjets_false
book_histos(df_TTBAR, 'CR_ttbar', h)
#### CR opposite sign 
df_OPPOSITESIGN = df_OS.Filter("BVeto_loose(Jet_pt, Jet_eta, Jet_btagDeepFlavB, GoodJets_idx) == true", "Bveto")
book_histos(df_OPPOSITESIGN, 'CR_opposite_sign', h)

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [14]:
#h["CR_fakes"]["Vgamma"]["Lepton_pt"]["etau"]
h["CR_fakes"]["Vgamma"]["Lepton_pt"]

{'etau': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x559a8d0e2590>,
 'mutau': <cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x559a8d04f560>}

In [15]:
h_varied = {}

for region, samples_dict in h.items():
    h_varied[region] = {}
    for sample, histos_dict in samples_dict.items(): 
        h_varied[region][sample] = {}
        for feature, histo in histos_dict.items():
            h_varied[region][sample][feature] = {}
            for final_state in ['etau', 'mutau']:
                h_varied[region][sample][feature][final_state] = ROOT.RDF.Experimental.VariationsFor(histo[final_state])

In [ ]:
h["CR_fakes"]["Vgamma"]["Lepton_pt"]["etau"].Draw()

In [ ]:
h_varied["CR_fakes"]["Vgamma"]["Lepton_pt"]["etau"]["nominal"].Draw()

# Save histos

In [ ]:
if results_dir not in os.listdir():
    os.mkdir("./" + results_dir)

In [ ]:
%%time
import math

if "plots" not in os.listdir("./" + results_dir):
    os.mkdir("./" + results_dir +"/" + "plots")
folder = "./"+ results_dir + "/plots"
if "electron" not in os.listdir(folder):
    os.mkdir(folder + "/" + "electron")
if "muon" not in os.listdir(folder):
    os.mkdir(folder + "/" + "muon")
for lepton, final_state in zip(['electron', 'muon'], ['etau', 'mutau']):
    for region, samples_dict in h.items():
        for sample, histos_dict in samples_dict.items(): 
            file = ROOT.TFile(folder + "/" + lepton + "/" + "{}.root".format(sample), 'UPDATE')
            for feature, histo in histos_dict.items():
                #histo[final_state].Write()
                h1 = histo[final_state].GetValue()
                nbins = h1.GetNbinsX()
                h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                if sample == "Fakes":
                    for bidx in range(nbins):          
                        bidx_l = bidx + 1
                        h1.SetBinError(bidx_l, 0.3*h1.GetBinContent(bidx_l))
                h1.Write()
            file.Close()

# Create and save stackplots

In [ ]:
if "stack" not in os.listdir("./" + results_dir):
    os.mkdir("./" + results_dir + "/stack")
for region in ["SR", "CR_fakes", "CR_ttbar", "CR_opposite_sign"]:
    for v in variables[region]:
        if region != "SR":
            stackplot(region, v, "etau", "./" + results_dir + "/stack", h)
            stackplot(region, v, "mutau", "./" + results_dir +"/stack", h) 
        else:
            stackplot(region, v, "etau", "./" + results_dir + "/stack", h, blinded = True)
            stackplot(region, v, "mutau", "./" + results_dir + "/stack", h, blinded = True) 

In [ ]:
h["CR_fakes"]["Vgamma"]["Lepton_pt"]["etau"].Draw()

In [ ]:
#ROOT.TFile.Open("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/tauSF/TauID_SF_pt_DeepTau2017v2p1VSjet_2017ReReco.root")

In [ ]:
#!xrdcp root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/ZG_UL2017/220227_181212/0000/tree_hadd_1.root .

In [ ]:
#columns = [ "run", "luminosityBlock", "event", "nboostedTau", "boostedTau_chargedIso", "boostedTau_eta", "boostedTau_leadTkDeltaEta", "boostedTau_leadTkDeltaPhi", "boostedTau_leadTkPtOverTauPt", "boostedTau_mass", "boostedTau_neutralIso", "boostedTau_phi", "boostedTau_photonsOutsideSignalCone", "boostedTau_pt", "boostedTau_puCorr", "boostedTau_rawAntiEle2018", "boostedTau_rawIso", "boostedTau_rawIsodR03", "boostedTau_rawMVAnewDM2017v2", "boostedTau_rawMVAoldDM2017v2", "boostedTau_rawMVAoldDMdR032017v2", "boostedTau_charge", "boostedTau_decayMode", "boostedTau_jetIdx", "boostedTau_rawAntiEleCat2018", "boostedTau_idAntiEle2018", "boostedTau_idAntiMu", "boostedTau_idMVAnewDM2017v2", "boostedTau_idMVAoldDM2017v2", "boostedTau_idMVAoldDMdR032017v2", "btagWeight_CSVV2", "btagWeight_DeepCSVB", "CaloMET_phi", "CaloMET_pt", "CaloMET_sumEt", "ChsMET_phi", "ChsMET_pt", "ChsMET_sumEt", "nCorrT1METJet", "CorrT1METJet_area", "CorrT1METJet_eta", "CorrT1METJet_muonSubtrFactor", "CorrT1METJet_phi", "CorrT1METJet_rawPt", "DeepMETResolutionTune_phi", "DeepMETResolutionTune_pt", "DeepMETResponseTune_phi", "DeepMETResponseTune_pt", "nElectron", "Electron_dEscaleDown", "Electron_dEscaleUp", "Electron_dEsigmaDown", "Electron_dEsigmaUp", "Electron_deltaEtaSC", "Electron_dr03EcalRecHitSumEt", "Electron_dr03HcalDepth1TowerSumEt", "Electron_dr03TkSumPt", "Electron_dr03TkSumPtHEEP", "Electron_dxy", "Electron_dxyErr", "Electron_dz", "Electron_dzErr", "Electron_eCorr", "Electron_eInvMinusPInv", "Electron_energyErr", "Electron_eta", "Electron_hoe", "Electron_ip3d", "Electron_jetPtRelv2", "Electron_jetRelIso", "Electron_mass", "Electron_miniPFRelIso_all", "Electron_miniPFRelIso_chg", "Electron_mvaFall17V2Iso", "Electron_mvaFall17V2noIso", "Electron_pfRelIso03_all", "Electron_pfRelIso03_chg", "Electron_phi", "Electron_pt", "Electron_r9", "Electron_scEtOverPt", "Electron_sieie", "Electron_sip3d", "Electron_mvaTTH", "Electron_charge", "Electron_cutBased", "Electron_jetIdx", "Electron_pdgId", "Electron_photonIdx", "Electron_tightCharge", "Electron_vidNestedWPBitmap", "Electron_vidNestedWPBitmapHEEP", "Electron_convVeto", "Electron_cutBased_HEEP", "Electron_isPFcand", "Electron_jetNDauCharged", "Electron_lostHits", "Electron_mvaFall17V2Iso_WP80", "Electron_mvaFall17V2Iso_WP90", "Electron_mvaFall17V2Iso_WPL", "Electron_mvaFall17V2noIso_WP80", "Electron_mvaFall17V2noIso_WP90", "Electron_mvaFall17V2noIso_WPL", "Electron_seedGain", "nFatJet", "FatJet_area", "FatJet_btagCSVV2", "FatJet_btagDDBvLV2", "FatJet_btagDDCvBV2", "FatJet_btagDDCvLV2", "FatJet_btagDeepB", "FatJet_btagHbb", "FatJet_deepTagMD_H4qvsQCD", "FatJet_deepTagMD_HbbvsQCD", "FatJet_deepTagMD_TvsQCD", "FatJet_deepTagMD_WvsQCD", "FatJet_deepTagMD_ZHbbvsQCD", "FatJet_deepTagMD_ZHccvsQCD", "FatJet_deepTagMD_ZbbvsQCD", "FatJet_deepTagMD_ZvsQCD", "FatJet_deepTagMD_bbvsLight", "FatJet_deepTagMD_ccvsLight", "FatJet_deepTag_H", "FatJet_deepTag_QCD", "FatJet_deepTag_QCDothers", "FatJet_deepTag_TvsQCD", "FatJet_deepTag_WvsQCD", "FatJet_deepTag_ZvsQCD", "FatJet_eta", "FatJet_mass", "FatJet_msoftdrop", "FatJet_n2b1", "FatJet_n3b1", "FatJet_particleNetMD_QCD", "FatJet_particleNetMD_Xbb", "FatJet_particleNetMD_Xcc", "FatJet_particleNetMD_Xqq", "FatJet_particleNet_H4qvsQCD", "FatJet_particleNet_HbbvsQCD", "FatJet_particleNet_HccvsQCD", "FatJet_particleNet_QCD", "FatJet_particleNet_TvsQCD", "FatJet_particleNet_WvsQCD", "FatJet_particleNet_ZvsQCD", "FatJet_particleNet_mass", "FatJet_phi", "FatJet_pt", "FatJet_rawFactor", "FatJet_tau1", "FatJet_tau2", "FatJet_tau3", "FatJet_tau4", "FatJet_lsf3", "FatJet_jetId", "FatJet_subJetIdx1", "FatJet_subJetIdx2", "FatJet_electronIdx3SJ", "FatJet_muonIdx3SJ", "FatJet_nConstituents", "nFsrPhoton", "FsrPhoton_dROverEt2", "FsrPhoton_eta", "FsrPhoton_phi", "FsrPhoton_pt", "FsrPhoton_relIso03", "FsrPhoton_muonIdx", "nGenJetAK8", "GenJetAK8_eta", "GenJetAK8_mass", "GenJetAK8_phi", "GenJetAK8_pt", "nGenJet", "GenJet_eta", "GenJet_mass", "GenJet_phi", "GenJet_pt", "nGenPart", "GenPart_eta", "GenPart_mass", "GenPart_phi", "GenPart_pt", "GenPart_genPartIdxMother", "GenPart_pdgId", "GenPart_status", "GenPart_statusFlags", "nSubGenJetAK8", "SubGenJetAK8_eta", "SubGenJetAK8_mass", "SubGenJetAK8_phi", "SubGenJetAK8_pt", "Generator_binvar", "Generator_scalePDF", "Generator_weight", "Generator_x1", "Generator_x2", "Generator_xpdf1", "Generator_xpdf2", "Generator_id1", "Generator_id2", "GenVtx_x", "GenVtx_y", "GenVtx_z", "nGenVisTau", "GenVisTau_eta", "GenVisTau_mass", "GenVisTau_phi", "GenVisTau_pt", "GenVisTau_charge", "GenVisTau_genPartIdxMother", "GenVisTau_status", "genWeight", "LHEWeight_originalXWGTUP", "nLHEPdfWeight", "LHEPdfWeight", "nLHEReweightingWeight", "LHEReweightingWeight", "nLHEScaleWeight", "LHEScaleWeight", "nPSWeight", "PSWeight", "nIsoTrack", "IsoTrack_dxy", "IsoTrack_dz", "IsoTrack_eta", "IsoTrack_pfRelIso03_all", "IsoTrack_pfRelIso03_chg", "IsoTrack_phi", "IsoTrack_pt", "IsoTrack_miniPFRelIso_all", "IsoTrack_miniPFRelIso_chg", "IsoTrack_charge", "IsoTrack_fromPV", "IsoTrack_pdgId", "IsoTrack_isHighPurityTrack", "IsoTrack_isPFcand", "IsoTrack_isFromLostTrack", "nJet", "Jet_area", "Jet_btagCSVV2", "Jet_btagDeepB", "Jet_btagDeepCvB", "Jet_btagDeepCvL", "Jet_btagDeepFlavB", "Jet_btagDeepFlavCvB", "Jet_btagDeepFlavCvL", "Jet_btagDeepFlavQG", "Jet_chEmEF", "Jet_chFPV0EF", "Jet_chHEF", "Jet_eta", "Jet_hfsigmaEtaEta", "Jet_hfsigmaPhiPhi", "Jet_mass", "Jet_muEF", "Jet_muonSubtrFactor", "Jet_neEmEF", "Jet_neHEF", "Jet_phi", "Jet_pt", "Jet_puIdDisc", "Jet_qgl", "Jet_rawFactor", "Jet_bRegCorr", "Jet_bRegRes", "Jet_cRegCorr", "Jet_cRegRes", "Jet_electronIdx1", "Jet_electronIdx2", "Jet_hfadjacentEtaStripsSize", "Jet_hfcentralEtaStripSize", "Jet_jetId", "Jet_muonIdx1", "Jet_muonIdx2", "Jet_nElectrons", "Jet_nMuons", "Jet_puId", "Jet_nConstituents", "L1PreFiringWeight_Dn", "L1PreFiringWeight_ECAL_Dn", "L1PreFiringWeight_ECAL_Nom", "L1PreFiringWeight_ECAL_Up", "L1PreFiringWeight_Muon_Nom", "L1PreFiringWeight_Muon_StatDn", "L1PreFiringWeight_Muon_StatUp", "L1PreFiringWeight_Muon_SystDn", "L1PreFiringWeight_Muon_SystUp", "L1PreFiringWeight_Nom", "L1PreFiringWeight_Up", "LHE_HT", "LHE_HTIncoming", "LHE_Vpt", "LHE_AlphaS", "LHE_Njets", "LHE_Nb", "LHE_Nc", "LHE_Nuds", "LHE_Nglu", "LHE_NpNLO", "LHE_NpLO", "nLHEPart", "LHEPart_pt", "LHEPart_eta", "LHEPart_phi", "LHEPart_mass", "LHEPart_incomingpz", "LHEPart_pdgId", "LHEPart_status", "LHEPart_spin", "nLowPtElectron", "LowPtElectron_ID", "LowPtElectron_convVtxRadius", "LowPtElectron_deltaEtaSC", "LowPtElectron_dxy", "LowPtElectron_dxyErr", "LowPtElectron_dz", "LowPtElectron_dzErr", "LowPtElectron_eInvMinusPInv", "LowPtElectron_embeddedID", "LowPtElectron_energyErr", "LowPtElectron_eta", "LowPtElectron_hoe", "LowPtElectron_mass", "LowPtElectron_miniPFRelIso_all", "LowPtElectron_miniPFRelIso_chg", "LowPtElectron_phi", "LowPtElectron_pt", "LowPtElectron_ptbiased", "LowPtElectron_r9", "LowPtElectron_scEtOverPt", "LowPtElectron_sieie", "LowPtElectron_unbiased", "LowPtElectron_charge", "LowPtElectron_convWP", "LowPtElectron_pdgId", "LowPtElectron_convVeto", "LowPtElectron_lostHits", "GenMET_phi", "GenMET_pt", "MET_MetUnclustEnUpDeltaX", "MET_MetUnclustEnUpDeltaY", "MET_covXX", "MET_covXY", "MET_covYY", "MET_phi", "MET_pt", "MET_significance", "MET_sumEt", "MET_sumPtUnclustered", "nMuon", "Muon_dxy", "Muon_dxyErr", "Muon_dxybs", "Muon_dz", "Muon_dzErr", "Muon_eta", "Muon_ip3d", "Muon_jetPtRelv2", "Muon_jetRelIso", "Muon_mass", "Muon_miniPFRelIso_all", "Muon_miniPFRelIso_chg", "Muon_pfRelIso03_all", "Muon_pfRelIso03_chg", "Muon_pfRelIso04_all", "Muon_phi", "Muon_pt", "Muon_ptErr", "Muon_segmentComp", "Muon_sip3d", "Muon_softMva", "Muon_tkRelIso", "Muon_tunepRelPt", "Muon_mvaLowPt", "Muon_mvaTTH", "Muon_charge", "Muon_jetIdx", "Muon_nStations", "Muon_nTrackerLayers", "Muon_pdgId", "Muon_tightCharge", "Muon_fsrPhotonIdx", "Muon_highPtId", "Muon_highPurity", "Muon_inTimeMuon", "Muon_isGlobal", "Muon_isPFcand", "Muon_isStandalone", "Muon_isTracker", "Muon_jetNDauCharged", "Muon_looseId", "Muon_mediumId", "Muon_mediumPromptId", "Muon_miniIsoId", "Muon_multiIsoId", "Muon_mvaId", "Muon_mvaLowPtId", "Muon_pfIsoId", "Muon_puppiIsoId", "Muon_softId", "Muon_softMvaId", "Muon_tightId", "Muon_tkIsoId", "Muon_triggerIdLoose", "nPhoton", "Photon_dEscaleDown", "Photon_dEscaleUp", "Photon_dEsigmaDown", "Photon_dEsigmaUp", "Photon_eCorr", "Photon_energyErr", "Photon_eta", "Photon_hoe", "Photon_mass", "Photon_mvaID", "Photon_mvaID_Fall17V1p1", "Photon_pfRelIso03_all", "Photon_pfRelIso03_chg", "Photon_phi", "Photon_pt", "Photon_r9", "Photon_sieie", "Photon_charge", "Photon_cutBased", "Photon_cutBased_Fall17V1Bitmap", "Photon_electronIdx", "Photon_jetIdx", "Photon_pdgId", "Photon_vidNestedWPBitmap", "Photon_electronVeto", "Photon_isScEtaEB", "Photon_isScEtaEE", "Photon_mvaID_WP80", "Photon_mvaID_WP90", "Photon_pixelSeed", "Photon_seedGain", "Pileup_nTrueInt", "Pileup_pudensity", "Pileup_gpudensity", "Pileup_nPU", "Pileup_sumEOOT", "Pileup_sumLOOT", "PuppiMET_phi", "PuppiMET_phiJERDown", "PuppiMET_phiJERUp", "PuppiMET_phiJESDown", "PuppiMET_phiJESUp", "PuppiMET_phiUnclusteredDown", "PuppiMET_phiUnclusteredUp", "PuppiMET_pt", "PuppiMET_ptJERDown", "PuppiMET_ptJERUp", "PuppiMET_ptJESDown", "PuppiMET_ptJESUp", "PuppiMET_ptUnclusteredDown", "PuppiMET_ptUnclusteredUp", "PuppiMET_sumEt", "RawMET_phi", "RawMET_pt", "RawMET_sumEt", "RawPuppiMET_phi", "RawPuppiMET_pt", "RawPuppiMET_sumEt", "nGenDressedLepton", "GenDressedLepton_eta", "GenDressedLepton_mass", "GenDressedLepton_phi", "GenDressedLepton_pt", "GenDressedLepton_pdgId", "GenDressedLepton_hasTauAnc", "nGenIsolatedPhoton", "GenIsolatedPhoton_eta", "GenIsolatedPhoton_mass", "GenIsolatedPhoton_phi", "GenIsolatedPhoton_pt", "nSoftActivityJet", "SoftActivityJet_eta", "SoftActivityJet_phi", "SoftActivityJet_pt", "SoftActivityJetHT", "SoftActivityJetHT10", "SoftActivityJetHT2", "SoftActivityJetHT5", "SoftActivityJetNjets10", "SoftActivityJetNjets2", "SoftActivityJetNjets5", "nSubJet", "SubJet_btagCSVV2", "SubJet_btagDeepB", "SubJet_eta", "SubJet_mass", "SubJet_n2b1", "SubJet_n3b1", "SubJet_phi", "SubJet_pt", "SubJet_rawFactor", "SubJet_tau1", "SubJet_tau2", "SubJet_tau3", "SubJet_tau4", "nTau", "Tau_chargedIso", "Tau_dxy", "Tau_dz", "Tau_eta", "Tau_leadTkDeltaEta", "Tau_leadTkDeltaPhi", "Tau_leadTkPtOverTauPt", "Tau_mass", "Tau_neutralIso", "Tau_phi", "Tau_photonsOutsideSignalCone", "Tau_pt", "Tau_puCorr", "Tau_rawDeepTau2017v2p1VSe", "Tau_rawDeepTau2017v2p1VSjet", "Tau_rawDeepTau2017v2p1VSmu", "Tau_rawIso", "Tau_rawIsodR03", "Tau_charge", "Tau_decayMode", "Tau_jetIdx", "Tau_idAntiEleDeadECal", "Tau_idAntiMu", "Tau_idDecayModeOldDMs", "Tau_idDeepTau2017v2p1VSe", "Tau_idDeepTau2017v2p1VSjet", "Tau_idDeepTau2017v2p1VSmu", "TkMET_phi", "TkMET_pt", "TkMET_sumEt", "nTrigObj", "TrigObj_pt", "TrigObj_eta", "TrigObj_phi", "TrigObj_l1pt", "TrigObj_l1pt_2", "TrigObj_l2pt", "TrigObj_id", "TrigObj_l1iso", "TrigObj_l1charge", "TrigObj_filterBits", "nOtherPV", "OtherPV_z", "PV_ndof", "PV_x", "PV_y", "PV_z", "PV_chi2", "PV_score", "PV_npvs", "PV_npvsGood", "nSV", "SV_dlen", "SV_dlenSig", "SV_dxy", "SV_dxySig", "SV_pAngle", "SV_charge", "boostedTau_genPartIdx", "boostedTau_genPartFlav", "Electron_genPartIdx", "Electron_genPartFlav", "FatJet_genJetAK8Idx", "FatJet_hadronFlavour", "FatJet_nBHadrons", "FatJet_nCHadrons", "GenJetAK8_partonFlavour", "GenJetAK8_hadronFlavour", "GenJet_partonFlavour", "GenJet_hadronFlavour", "GenVtx_t0", "Jet_genJetIdx", "Jet_hadronFlavour", "Jet_partonFlavour", "LowPtElectron_genPartIdx", "LowPtElectron_genPartFlav", "Muon_genPartIdx", "Muon_genPartFlav", "Photon_genPartIdx", "Photon_genPartFlav", "MET_fiducialGenPhi", "MET_fiducialGenPt", "Electron_cleanmask", "Jet_cleanmask", "Muon_cleanmask", "Photon_cleanmask", "Tau_cleanmask", "SubJet_hadronFlavour", "SubJet_nBHadrons", "SubJet_nCHadrons", "SV_chi2", "SV_eta", "SV_mass", "SV_ndof", "SV_phi", "SV_pt", "SV_x", "SV_y", "SV_z", "SV_ntracks", "Tau_genPartIdx", "Tau_genPartFlav", "L1_AlwaysTrue", "L1_BPTX_AND_Ref1_VME", "L1_BPTX_AND_Ref3_VME", "L1_BPTX_AND_Ref4_VME", "L1_BPTX_BeamGas_B1_VME", "L1_BPTX_BeamGas_B2_VME", "L1_BPTX_BeamGas_Ref1_VME", "L1_BPTX_BeamGas_Ref2_VME", "L1_BPTX_NotOR_VME", "L1_BPTX_OR_Ref3_VME", "L1_BPTX_OR_Ref4_VME", "L1_BPTX_RefAND_VME", "L1_BptxMinus", "L1_BptxOR", "L1_BptxPlus", "L1_BptxXOR", "L1_CDC_SingleMu_3_er1p2_TOP120_DPHI2p618_3p142", "L1_DoubleEG6_HTT240er", "L1_DoubleEG6_HTT250er", "L1_DoubleEG6_HTT255er", "L1_DoubleEG6_HTT270er", "L1_DoubleEG6_HTT300er", "L1_DoubleEG8er2p6_HTT255er", "L1_DoubleEG8er2p6_HTT270er", "L1_DoubleEG8er2p6_HTT300er", "L1_DoubleEG_15_10", "L1_DoubleEG_18_17", "L1_DoubleEG_20_18", "L1_DoubleEG_22_10", "L1_DoubleEG_22_12", "L1_DoubleEG_22_15", "L1_DoubleEG_23_10", "L1_DoubleEG_24_17", "L1_DoubleEG_25_12", "L1_DoubleEG_25_13", "L1_DoubleEG_25_14", "L1_DoubleEG_LooseIso23_10", "L1_DoubleEG_LooseIso24_10", "L1_DoubleIsoTau28er2p1", "L1_DoubleIsoTau30er2p1", "L1_DoubleIsoTau32er2p1", "L1_DoubleIsoTau33er2p1", "L1_DoubleIsoTau34er2p1", "L1_DoubleIsoTau35er2p1", "L1_DoubleIsoTau36er2p1", "L1_DoubleIsoTau38er2p1", "L1_DoubleJet100er2p3_dEta_Max1p6", "L1_DoubleJet100er2p7", "L1_DoubleJet112er2p3_dEta_Max1p6", "L1_DoubleJet112er2p7", "L1_DoubleJet120er2p7", "L1_DoubleJet150er2p7", "L1_DoubleJet30_Mass_Min300_dEta_Max1p5", "L1_DoubleJet30_Mass_Min320_dEta_Max1p5", "L1_DoubleJet30_Mass_Min340_dEta_Max1p5", "L1_DoubleJet30_Mass_Min360_dEta_Max1p5", "L1_DoubleJet30_Mass_Min380_dEta_Max1p5", "L1_DoubleJet30_Mass_Min400_Mu10", "L1_DoubleJet30_Mass_Min400_Mu6", "L1_DoubleJet30_Mass_Min400_dEta_Max1p5", "L1_DoubleJet35_rmovlp_IsoTau45_Mass_Min450", "L1_DoubleJet40er2p7", "L1_DoubleJet50er2p7", "L1_DoubleJet60er2p7", "L1_DoubleJet60er2p7_ETM100", "L1_DoubleJet60er2p7_ETM60", "L1_DoubleJet60er2p7_ETM70", "L1_DoubleJet60er2p7_ETM80", "L1_DoubleJet60er2p7_ETM90", "L1_DoubleJet80er2p7", "L1_DoubleJet_100_30_DoubleJet30_Mass_Min620", "L1_DoubleJet_100_35_DoubleJet35_Mass_Min620", "L1_DoubleJet_110_35_DoubleJet35_Mass_Min620", "L1_DoubleJet_110_40_DoubleJet40_Mass_Min620", "L1_DoubleJet_115_35_DoubleJet35_Mass_Min620", "L1_DoubleJet_115_40_DoubleJet40_Mass_Min620", "L1_DoubleJet_90_30_DoubleJet30_Mass_Min620", "L1_DoubleLooseIsoEG22er2p1", "L1_DoubleLooseIsoEG24er2p1", "L1_DoubleMu0", "L1_DoubleMu0_ETM40", "L1_DoubleMu0_ETM55", "L1_DoubleMu0_ETM60", "L1_DoubleMu0_ETM65", "L1_DoubleMu0_ETM70", "L1_DoubleMu0_SQ", "L1_DoubleMu0_SQ_OS", "L1_DoubleMu0er1p4_SQ_OS_dR_Max1p4", "L1_DoubleMu0er1p4_dEta_Max1p8_OS", "L1_DoubleMu0er1p5_SQ_OS", "L1_DoubleMu0er1p5_SQ_OS_dR_Max1p4", "L1_DoubleMu0er1p5_SQ_dR_Max1p4", "L1_DoubleMu0er2_SQ_dR_Max1p4", "L1_DoubleMu18er2p1", "L1_DoubleMu22er2p1", "L1_DoubleMu3_OS_DoubleEG7p5Upsilon", "L1_DoubleMu3_SQ_ETMHF40_Jet60_OR_DoubleJet30", "L1_DoubleMu3_SQ_ETMHF50_Jet60_OR_DoubleJet30", "L1_DoubleMu3_SQ_ETMHF60_Jet60_OR_DoubleJet30", "L1_DoubleMu3_SQ_ETMHF70_Jet60_OR_DoubleJet30", "L1_DoubleMu3_SQ_ETMHF80_Jet60_OR_DoubleJet30", "L1_DoubleMu3_SQ_HTT100er", "L1_DoubleMu3_SQ_HTT200er", "L1_DoubleMu3_SQ_HTT220er", "L1_DoubleMu3_SQ_HTT240er", "L1_DoubleMu4_OS_EG12", "L1_DoubleMu4_SQ_OS", "L1_DoubleMu4_SQ_OS_dR_Max1p2", "L1_DoubleMu4p5_SQ", "L1_DoubleMu4p5_SQ_OS", "L1_DoubleMu4p5_SQ_OS_dR_Max1p2", "L1_DoubleMu4p5er2p0_SQ_OS", "L1_DoubleMu4p5er2p0_SQ_OS_Mass7to18", "L1_DoubleMu5Upsilon_OS_DoubleEG3", "L1_DoubleMu5_OS_EG12", "L1_DoubleMu5_SQ_OS", "L1_DoubleMu5_SQ_OS_Mass7to18", "L1_DoubleMu6_SQ_OS", "L1_DoubleMu7_EG7", "L1_DoubleMu7_SQ_EG7", "L1_DoubleMu8_SQ", "L1_DoubleMu_10_0_dEta_Max1p8", "L1_DoubleMu_11_4", "L1_DoubleMu_12_5", "L1_DoubleMu_12_8", "L1_DoubleMu_13_6", "L1_DoubleMu_15_5", "L1_DoubleMu_15_5_SQ", "L1_DoubleMu_15_7", "L1_DoubleMu_15_7_SQ", "L1_DoubleMu_15_7_SQ_Mass_Min4", "L1_DoubleMu_20_2_SQ_Mass_Max20", "L1_DoubleTau50er2p1", "L1_DoubleTau70er2p1", "L1_EG25er2p1_HTT125er", "L1_EG27er2p1_HTT200er", "L1_ETM100", "L1_ETM100_Jet60_dPhi_Min0p4", "L1_ETM105", "L1_ETM110", "L1_ETM110_Jet60_dPhi_Min0p4", "L1_ETM115", "L1_ETM120", "L1_ETM150", "L1_ETM30", "L1_ETM40", "L1_ETM50", "L1_ETM60", "L1_ETM70", "L1_ETM75", "L1_ETM75_Jet60_dPhi_Min0p4", "L1_ETM80", "L1_ETM80_Jet60_dPhi_Min0p4", "L1_ETM85", "L1_ETM90", "L1_ETM90_Jet60_dPhi_Min0p4", "L1_ETM95", "L1_ETMHF100", "L1_ETMHF100_HTT60er", "L1_ETMHF100_Jet60_OR_DiJet30woTT28", "L1_ETMHF100_Jet60_OR_DoubleJet30", "L1_ETMHF100_Jet90_OR_DoubleJet45_OR_TripleJet30", "L1_ETMHF110", "L1_ETMHF110_HTT60er", "L1_ETMHF110_Jet60_OR_DiJet30woTT28", "L1_ETMHF110_Jet90_OR_DoubleJet45_OR_TripleJet30", "L1_ETMHF120", "L1_ETMHF120_HTT60er", "L1_ETMHF120_Jet60_OR_DiJet30woTT28", "L1_ETMHF150", "L1_ETMHF70", "L1_ETMHF70_Jet90_OR_DoubleJet45_OR_TripleJet30", "L1_ETMHF80", "L1_ETMHF80_HTT60er", "L1_ETMHF80_Jet90_OR_DoubleJet45_OR_TripleJet30", "L1_ETMHF90", "L1_ETMHF90_HTT60er", "L1_ETMHF90_Jet90_OR_DoubleJet45_OR_TripleJet30", "L1_ETT100_BptxAND", "L1_ETT110_BptxAND", "L1_ETT40_BptxAND", "L1_ETT50_BptxAND", "L1_ETT60_BptxAND", "L1_ETT70_BptxAND", "L1_ETT75_BptxAND", "L1_ETT80_BptxAND", "L1_ETT85_BptxAND", "L1_ETT90_BptxAND", "L1_ETT95_BptxAND", "L1_FirstBunchAfterTrain", "L1_FirstBunchInTrain", "L1_FirstCollisionInOrbit", "L1_FirstCollisionInTrain", "L1_HTT120er", "L1_HTT160er", "L1_HTT200er", "L1_HTT220er", "L1_HTT240er", "L1_HTT250er_QuadJet_70_55_40_35_er2p5", "L1_HTT255er", "L1_HTT270er", "L1_HTT280er", "L1_HTT280er_QuadJet_70_55_40_35_er2p5", "L1_HTT300er", "L1_HTT300er_QuadJet_70_55_40_35_er2p5", "L1_HTT320er", "L1_HTT320er_QuadJet_70_55_40_40_er2p4", "L1_HTT320er_QuadJet_70_55_40_40_er2p5", "L1_HTT320er_QuadJet_70_55_45_45_er2p5", "L1_HTT340er", "L1_HTT340er_QuadJet_70_55_40_40_er2p5", "L1_HTT340er_QuadJet_70_55_45_45_er2p5", "L1_HTT380er", "L1_HTT400er", "L1_HTT450er", "L1_HTT500er", "L1_IsoEG33_Mt40", "L1_IsoEG33_Mt44", "L1_IsoEG33_Mt48", "L1_IsoTau40er_ETM100", "L1_IsoTau40er_ETM105", "L1_IsoTau40er_ETM110", "L1_IsoTau40er_ETM115", "L1_IsoTau40er_ETM120", "L1_IsoTau40er_ETM80", "L1_IsoTau40er_ETM85", "L1_IsoTau40er_ETM90", "L1_IsoTau40er_ETM95", "L1_IsoTau40er_ETMHF100", "L1_IsoTau40er_ETMHF110", "L1_IsoTau40er_ETMHF120", "L1_IsoTau40er_ETMHF80", "L1_IsoTau40er_ETMHF90", "L1_IsolatedBunch", "L1_LastCollisionInTrain", "L1_LooseIsoEG22er2p1_IsoTau26er2p1_dR_Min0p3", "L1_LooseIsoEG24er2p1_HTT100er", "L1_LooseIsoEG24er2p1_IsoTau27er2p1_dR_Min0p3", "L1_LooseIsoEG24er2p1_Jet26er2p7_dR_Min0p3", "L1_LooseIsoEG24er2p1_TripleJet_26er2p7_26_26er2p7", "L1_LooseIsoEG26er2p1_HTT100er", "L1_LooseIsoEG26er2p1_Jet34er2p7_dR_Min0p3", "L1_LooseIsoEG28er2p1_HTT100er", "L1_LooseIsoEG28er2p1_Jet34er2p7_dR_Min0p3", "L1_LooseIsoEG30er2p1_Jet34er2p7_dR_Min0p3", "L1_MU20_EG15", "L1_MinimumBiasHF0_AND_BptxAND", "L1_MinimumBiasHF0_OR_BptxAND", "L1_Mu10er2p1_ETM30", "L1_Mu10er2p3_Jet32er2p3_dR_Max0p4_DoubleJet32er2p3_dEta_Max1p6", "L1_Mu12_EG10", "L1_Mu12er2p3_Jet40er2p3_dR_Max0p4_DoubleJet40er2p3_dEta_Max1p6", "L1_Mu14er2p1_ETM30", "L1_Mu15_HTT100er", "L1_Mu18_HTT100er", "L1_Mu18_Jet24er2p7", "L1_Mu18er2p1_IsoTau26er2p1", "L1_Mu18er2p1_Tau24er2p1", "L1_Mu20_EG10", "L1_Mu20_EG17", "L1_Mu20_LooseIsoEG6", "L1_Mu20er2p1_IsoTau26er2p1", "L1_Mu20er2p1_IsoTau27er2p1", "L1_Mu22er2p1_IsoTau28er2p1", "L1_Mu22er2p1_IsoTau30er2p1", "L1_Mu22er2p1_IsoTau32er2p1", "L1_Mu22er2p1_IsoTau33er2p1", "L1_Mu22er2p1_IsoTau34er2p1", "L1_Mu22er2p1_IsoTau35er2p1", "L1_Mu22er2p1_IsoTau36er2p1", "L1_Mu22er2p1_IsoTau38er2p1", "L1_Mu22er2p1_IsoTau40er2p1", "L1_Mu22er2p1_Tau50er2p1", "L1_Mu22er2p1_Tau70er2p1", "L1_Mu23_EG10", "L1_Mu23_LooseIsoEG10", "L1_Mu3_Jet120er2p7_dEta_Max0p4_dPhi_Max0p4", "L1_Mu3_Jet16er2p7_dEta_Max0p4_dPhi_Max0p4", "L1_Mu3_Jet30er2p5", "L1_Mu3_Jet60er2p7_dEta_Max0p4_dPhi_Max0p4", "L1_Mu5_EG15", "L1_Mu5_EG20", "L1_Mu5_EG23", "L1_Mu5_LooseIsoEG18", "L1_Mu5_LooseIsoEG20", "L1_Mu6_DoubleEG10", "L1_Mu6_DoubleEG17", "L1_Mu6_HTT200er", "L1_Mu6_HTT240er", "L1_Mu6_HTT250er", "L1_Mu7_EG23", "L1_Mu7_LooseIsoEG20", "L1_Mu7_LooseIsoEG23", "L1_Mu8_HTT150er", "L1_NotBptxOR", "L1_QuadJet36er2p7_IsoTau52er2p1", "L1_QuadJet36er2p7_Tau52", "L1_QuadJet40er2p7", "L1_QuadJet50er2p7", "L1_QuadJet60er2p7", "L1_QuadMu0", "L1_SingleEG10", "L1_SingleEG15", "L1_SingleEG18", "L1_SingleEG24", "L1_SingleEG26", "L1_SingleEG28", "L1_SingleEG2_BptxAND", "L1_SingleEG30", "L1_SingleEG32", "L1_SingleEG34", "L1_SingleEG34er2p1", "L1_SingleEG36", "L1_SingleEG36er2p1", "L1_SingleEG38", "L1_SingleEG38er2p1", "L1_SingleEG40", "L1_SingleEG42", "L1_SingleEG45", "L1_SingleEG5", "L1_SingleEG50", "L1_SingleIsoEG18", "L1_SingleIsoEG18er2p1", "L1_SingleIsoEG20", "L1_SingleIsoEG20er2p1", "L1_SingleIsoEG22", "L1_SingleIsoEG22er2p1", "L1_SingleIsoEG24", "L1_SingleIsoEG24er2p1", "L1_SingleIsoEG26", "L1_SingleIsoEG26er2p1", "L1_SingleIsoEG28", "L1_SingleIsoEG28er2p1", "L1_SingleIsoEG30", "L1_SingleIsoEG30er2p1", "L1_SingleIsoEG32", "L1_SingleIsoEG32er2p1", "L1_SingleIsoEG33er2p1", "L1_SingleIsoEG34", "L1_SingleIsoEG34er2p1", "L1_SingleIsoEG35", "L1_SingleIsoEG35er2p1", "L1_SingleIsoEG36", "L1_SingleIsoEG36er2p1", "L1_SingleIsoEG37", "L1_SingleIsoEG38", "L1_SingleIsoEG38er2p1", "L1_SingleIsoEG40", "L1_SingleIsoEG40er2p1", "L1_SingleJet120", "L1_SingleJet120_FWD", "L1_SingleJet12_BptxAND", "L1_SingleJet140", "L1_SingleJet150", "L1_SingleJet16", "L1_SingleJet160", "L1_SingleJet170", "L1_SingleJet180", "L1_SingleJet20", "L1_SingleJet200", "L1_SingleJet20er2p7_NotBptxOR", "L1_SingleJet20er2p7_NotBptxOR_3BX", "L1_SingleJet35", "L1_SingleJet35_FWD", "L1_SingleJet35_HFm", "L1_SingleJet35_HFp", "L1_SingleJet43er2p7_NotBptxOR_3BX", "L1_SingleJet46er2p7_NotBptxOR_3BX", "L1_SingleJet60", "L1_SingleJet60_FWD", "L1_SingleJet60_HFm", "L1_SingleJet60_HFp", "L1_SingleJet90", "L1_SingleJet90_FWD", "L1_SingleMu0_BMTF", "L1_SingleMu0_EMTF", "L1_SingleMu0_OMTF", "L1_SingleMu10_LowQ", "L1_SingleMu11_LowQ", "L1_SingleMu12_LowQ_BMTF", "L1_SingleMu12_LowQ_EMTF", "L1_SingleMu12_LowQ_OMTF", "L1_SingleMu14er2p1", "L1_SingleMu16", "L1_SingleMu16er2p1", "L1_SingleMu18", "L1_SingleMu18er2p1", "L1_SingleMu20", "L1_SingleMu20er2p1", "L1_SingleMu22", "L1_SingleMu22_BMTF", "L1_SingleMu22_EMTF", "L1_SingleMu22_OMTF", "L1_SingleMu22er2p1", "L1_SingleMu25", "L1_SingleMu3", "L1_SingleMu30", "L1_SingleMu5", "L1_SingleMu7", "L1_SingleMuCosmics", "L1_SingleMuCosmics_BMTF", "L1_SingleMuCosmics_EMTF", "L1_SingleMuCosmics_OMTF", "L1_SingleMuOpen", "L1_SingleMuOpen_NotBptxOR", "L1_SingleMuOpen_NotBptxOR_3BX", "L1_SingleTau100er2p1", "L1_SingleTau120er2p1", "L1_SingleTau130er2p1", "L1_SingleTau140er2p1", "L1_SingleTau20", "L1_SingleTau80er2p1", "L1_TripleEG_14_10_8", "L1_TripleEG_18_17_8", "L1_TripleEG_LooseIso20_10_5", "L1_TripleJet_100_85_72_VBF", "L1_TripleJet_105_85_76_VBF", "L1_TripleJet_84_68_48_VBF", "L1_TripleJet_88_72_56_VBF", "L1_TripleJet_92_76_64_VBF", "L1_TripleJet_98_83_71_VBF", "L1_TripleMu0", "L1_TripleMu0_OQ", "L1_TripleMu3", "L1_TripleMu3_SQ", "L1_TripleMu_4_4_4", "L1_TripleMu_5OQ_3p5OQ_2p5OQ_DoubleMu_5_2p5_OQ_OS_Mass_5to17", "L1_TripleMu_5OQ_3p5OQ_2p5OQ_DoubleMu_5_2p5_OQ_OS_Mass_8to14", "L1_TripleMu_5SQ_3SQ_0OQ", "L1_TripleMu_5SQ_3SQ_0OQ_DoubleMu_5_3_SQ_OS_Mass_Max9", "L1_TripleMu_5SQ_3SQ_0_DoubleMu_5_3_SQ_OS_Mass_Max9", "L1_TripleMu_5_0_0", "L1_TripleMu_5_3_3", "L1_TripleMu_5_3p5_2p5", "L1_TripleMu_5_3p5_2p5_DoubleMu_5_2p5_OS_Mass_5to17", "L1_TripleMu_5_4_2p5_DoubleMu_5_2p5_OS_Mass_5to17", "L1_TripleMu_5_5_3", "L1_UnpairedBunchBptxMinus", "L1_UnpairedBunchBptxPlus", "L1_ZeroBias", "L1_ZeroBias_copy", "L1_UnprefireableEvent", "Flag_HBHENoiseFilter", "Flag_HBHENoiseIsoFilter", "Flag_CSCTightHaloFilter", "Flag_CSCTightHaloTrkMuUnvetoFilter", "Flag_CSCTightHalo2015Filter", "Flag_globalTightHalo2016Filter", "Flag_globalSuperTightHalo2016Filter", "Flag_HcalStripHaloFilter", "Flag_hcalLaserEventFilter", "Flag_EcalDeadCellTriggerPrimitiveFilter", "Flag_EcalDeadCellBoundaryEnergyFilter", "Flag_ecalBadCalibFilter", "Flag_goodVertices", "Flag_eeBadScFilter", "Flag_ecalLaserCorrFilter", "Flag_trkPOGFilters", "Flag_chargedHadronTrackResolutionFilter", "Flag_muonBadTrackFilter", "Flag_BadChargedCandidateFilter", "Flag_BadPFMuonFilter", "Flag_BadPFMuonDzFilter", "Flag_hfNoisyHitsFilter", "Flag_BadChargedCandidateSummer16Filter", "Flag_BadPFMuonSummer16Filter", "Flag_trkPOG_manystripclus53X", "Flag_trkPOG_toomanystripclus53X", "Flag_trkPOG_logErrorTooManyClusters", "Flag_METFilters", "L1Reco_step", "HLTriggerFirstPath", "HLT_Trimuon5_3p5_2_Upsilon_Muon", "HLT_Ele27_Ele37_CaloIdL_MW", "HLT_Mu27_Ele37_CaloIdL_MW", "HLT_Mu37_Ele27_CaloIdL_MW", "HLT_Mu37_TkMu27", "HLT_Mu3_PFJet40", "HLT_Mu7p5_L2Mu2_Jpsi", "HLT_Mu7p5_L2Mu2_Upsilon", "HLT_Mu7p5_Track2_Jpsi", "HLT_Mu7p5_Track3p5_Jpsi", "HLT_Mu7p5_Track7_Jpsi", "HLT_Mu7p5_Track2_Upsilon", "HLT_Mu7p5_Track3p5_Upsilon", "HLT_Mu7p5_Track7_Upsilon", "HLT_Ele20_WPTight_Gsf", "HLT_Ele20_WPLoose_Gsf", "HLT_Ele20_eta2p1_WPLoose_Gsf", "HLT_Ele27_WPTight_Gsf", "HLT_Ele32_WPTight_Gsf", "HLT_Ele35_WPTight_Gsf", "HLT_Ele35_WPTight_Gsf_L1EGMT", "HLT_Ele38_WPTight_Gsf", "HLT_Ele40_WPTight_Gsf", "HLT_Ele32_WPTight_Gsf_L1DoubleEG", "HLT_HT450_Beamspot", "HLT_HT300_Beamspot", "HLT_IsoMu20_eta2p1_LooseChargedIsoPFTau27_eta2p1_CrossL1", "HLT_IsoMu20_eta2p1_MediumChargedIsoPFTau27_eta2p1_CrossL1", "HLT_IsoMu20_eta2p1_TightChargedIsoPFTau27_eta2p1_CrossL1", "HLT_IsoMu20_eta2p1_LooseChargedIsoPFTau27_eta2p1_TightID_CrossL1", "HLT_IsoMu20_eta2p1_MediumChargedIsoPFTau27_eta2p1_TightID_CrossL1", "HLT_IsoMu20_eta2p1_TightChargedIsoPFTau27_eta2p1_TightID_CrossL1", "HLT_IsoMu24_eta2p1_LooseChargedIsoPFTau20_SingleL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau20_SingleL1", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau20_SingleL1", "HLT_IsoMu24_eta2p1_LooseChargedIsoPFTau20_TightID_SingleL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau20_TightID_SingleL1", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau20_TightID_SingleL1", "HLT_IsoMu20", "HLT_IsoMu24", "HLT_IsoMu24_eta2p1", "HLT_IsoMu27", "HLT_IsoMu30", "HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL", "HLT_Mu19_TrkIsoVVL_Mu9_TrkIsoVVL", "HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ", "HLT_Mu19_TrkIsoVVL_Mu9_TrkIsoVVL_DZ", "HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass8", "HLT_Mu19_TrkIsoVVL_Mu9_TrkIsoVVL_DZ_Mass8", "HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8", "HLT_Mu19_TrkIsoVVL_Mu9_TrkIsoVVL_DZ_Mass3p8", "HLT_Mu25_TkMu0_Onia", "HLT_Mu30_TkMu0_Onia", "HLT_Mu20_TkMu0_Phi", "HLT_Mu25_TkMu0_Phi", "HLT_Mu20", "HLT_Mu27", "HLT_Mu50", "HLT_Mu55", "HLT_OldMu100", "HLT_TkMu100", "HLT_PFJet40", "HLT_PFJet60", "HLT_PFJet80", "HLT_PFJet140", "HLT_PFJet200", "HLT_PFJet260", "HLT_PFJet320", "HLT_PFJet400", "HLT_PFJet450", "HLT_PFJet500", "HLT_PFJet550", "HLT_PFJetFwd40", "HLT_PFJetFwd60", "HLT_PFJetFwd80", "HLT_PFJetFwd140", "HLT_PFJetFwd200", "HLT_PFJetFwd260", "HLT_PFJetFwd320", "HLT_PFJetFwd400", "HLT_PFJetFwd450", "HLT_PFJetFwd500", "HLT_PFHT180", "HLT_PFHT250", "HLT_PFHT370", "HLT_PFHT430", "HLT_PFHT510", "HLT_PFHT590", "HLT_PFHT680", "HLT_PFHT780", "HLT_PFHT890", "HLT_PFHT1050", "HLT_PFHT500_PFMET100_PFMHT100_IDTight", "HLT_PFHT500_PFMET110_PFMHT110_IDTight", "HLT_PFHT700_PFMET85_PFMHT85_IDTight", "HLT_PFHT700_PFMET95_PFMHT95_IDTight", "HLT_PFHT800_PFMET75_PFMHT75_IDTight", "HLT_PFHT800_PFMET85_PFMHT85_IDTight", "HLT_MonoCentralPFJet80_PFMETNoMu110_PFMHTNoMu110_IDTight", "HLT_MonoCentralPFJet80_PFMETNoMu120_PFMHTNoMu120_IDTight", "HLT_MonoCentralPFJet80_PFMETNoMu130_PFMHTNoMu130_IDTight", "HLT_MonoCentralPFJet80_PFMETNoMu140_PFMHTNoMu140_IDTight", "HLT_MET105_IsoTrk50", "HLT_MET120_IsoTrk50", "HLT_SingleJet30_Mu12_SinglePFJet40", "HLT_Mu12_DoublePFJets40_CaloBTagCSV_p33", "HLT_Mu12_DoublePFJets100_CaloBTagCSV_p33", "HLT_Mu12_DoublePFJets200_CaloBTagCSV_p33", "HLT_Mu12_DoublePFJets350_CaloBTagCSV_p33", "HLT_Mu12_DoublePFJets40MaxDeta1p6_DoubleCaloBTagCSV_p33", "HLT_Mu12_DoublePFJets54MaxDeta1p6_DoubleCaloBTagCSV_p33", "HLT_Mu12_DoublePFJets62MaxDeta1p6_DoubleCaloBTagCSV_p33", "HLT_Photon300_NoHE", "HLT_Mu8_TrkIsoVVL", "HLT_Mu8_DiEle12_CaloIdL_TrackIdL_DZ", "HLT_Mu8_DiEle12_CaloIdL_TrackIdL", "HLT_Mu8_Ele8_CaloIdM_TrackIdM_Mass8_PFHT350_DZ", "HLT_Mu8_Ele8_CaloIdM_TrackIdM_Mass8_PFHT350", "HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ", "HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL", "HLT_Mu17_TrkIsoVVL", "HLT_Mu19_TrkIsoVVL", "HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL_DZ", "HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL", "HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL_DZ", "HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL", "HLT_Mu12_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL", "HLT_Mu12_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ", "HLT_Mu12_DoublePhoton20", "HLT_TriplePhoton_20_20_20_CaloIdLV2", "HLT_TriplePhoton_20_20_20_CaloIdLV2_R9IdVL", "HLT_TriplePhoton_30_30_10_CaloIdLV2", "HLT_TriplePhoton_30_30_10_CaloIdLV2_R9IdVL", "HLT_TriplePhoton_35_35_5_CaloIdLV2_R9IdVL", "HLT_Photon25", "HLT_Photon33", "HLT_Photon50", "HLT_Photon75", "HLT_Photon90", "HLT_Photon120", "HLT_Photon150", "HLT_Photon175", "HLT_Photon200", "HLT_Photon50_R9Id90_HE10_IsoM", "HLT_Photon75_R9Id90_HE10_IsoM", "HLT_Photon90_R9Id90_HE10_IsoM", "HLT_Photon120_R9Id90_HE10_IsoM", "HLT_Photon165_R9Id90_HE10_IsoM", "HLT_Photon90_CaloIdL_PFHT700", "HLT_TripleMu_5_3_3_Mass3p8to60_DZ", "HLT_TripleMu_10_5_5_DZ", "HLT_TripleMu_12_10_5", "HLT_Tau3Mu_Mu7_Mu1_TkMu1_Tau15", "HLT_Tau3Mu_Mu7_Mu1_TkMu1_Tau15_Charge1", "HLT_Tau3Mu_Mu7_Mu1_TkMu1_IsoTau15", "HLT_Tau3Mu_Mu7_Mu1_TkMu1_IsoTau15_Charge1", "HLT_Mu43NoFiltersNoVtx_Photon43_CaloIdL", "HLT_Mu48NoFiltersNoVtx_Photon48_CaloIdL", "HLT_HT425", "HLT_HT430_DisplacedDijet40_DisplacedTrack", "HLT_HT430_DisplacedDijet60_DisplacedTrack", "HLT_HT430_DisplacedDijet80_DisplacedTrack", "HLT_HT400_DisplacedDijet40_DisplacedTrack", "HLT_HT650_DisplacedDijet60_Inclusive", "HLT_HT550_DisplacedDijet80_Inclusive", "HLT_HT550_DisplacedDijet60_Inclusive", "HLT_HT650_DisplacedDijet80_Inclusive", "HLT_HT750_DisplacedDijet80_Inclusive", "HLT_TripleJet110_35_35_Mjj650_PFMET110", "HLT_TripleJet110_35_35_Mjj650_PFMET120", "HLT_TripleJet110_35_35_Mjj650_PFMET130", "HLT_Ele30_eta2p1_WPTight_Gsf_CentralPFJet35_EleCleaned", "HLT_Ele28_eta2p1_WPTight_Gsf_HT150", "HLT_Ele28_HighEta_SC20_Mass55", "HLT_Ele15_IsoVVVL_PFHT450_CaloBTagCSV_4p5", "HLT_Ele15_IsoVVVL_PFHT450_PFMET50", "HLT_Ele15_IsoVVVL_PFHT450", "HLT_Ele50_IsoVVVL_PFHT450", "HLT_Ele15_IsoVVVL_PFHT600", "HLT_Mu8_TrkIsoVVL_DiPFJet40_DEta3p5_MJJ750_HTT300_PFMETNoMu60", "HLT_Mu10_TrkIsoVVL_DiPFJet40_DEta3p5_MJJ750_HTT350_PFMETNoMu60", "HLT_Mu15_IsoVVVL_PFHT450_CaloBTagCSV_4p5", "HLT_Mu15_IsoVVVL_PFHT450_PFMET50", "HLT_Mu15_IsoVVVL_PFHT450", "HLT_Mu50_IsoVVVL_PFHT450", "HLT_Mu15_IsoVVVL_PFHT600", "HLT_TrkMu12_DoubleTrkMu5NoFiltersNoVtx", "HLT_TrkMu16_DoubleTrkMu6NoFiltersNoVtx", "HLT_TrkMu17_DoubleTrkMu8NoFiltersNoVtx", "HLT_Mu8", "HLT_Mu17", "HLT_Mu19", "HLT_Mu17_Photon30_IsoCaloId", "HLT_Ele8_CaloIdL_TrackIdL_IsoVL_PFJet30", "HLT_Ele12_CaloIdL_TrackIdL_IsoVL_PFJet30", "HLT_Ele23_CaloIdL_TrackIdL_IsoVL_PFJet30", "HLT_Ele8_CaloIdM_TrackIdM_PFJet30", "HLT_Ele17_CaloIdM_TrackIdM_PFJet30", "HLT_Ele23_CaloIdM_TrackIdM_PFJet30", "HLT_Ele50_CaloIdVT_GsfTrkIdT_PFJet165", "HLT_Ele115_CaloIdVT_GsfTrkIdT", "HLT_Ele135_CaloIdVT_GsfTrkIdT", "HLT_Ele145_CaloIdVT_GsfTrkIdT", "HLT_Ele200_CaloIdVT_GsfTrkIdT", "HLT_Ele250_CaloIdVT_GsfTrkIdT", "HLT_Ele300_CaloIdVT_GsfTrkIdT", "HLT_PFHT300PT30_QuadPFJet_75_60_45_40", "HLT_PFHT300PT30_QuadPFJet_75_60_45_40_TriplePFBTagCSV_3p0", "HLT_PFHT380_SixPFJet32_DoublePFBTagCSV_2p2", "HLT_PFHT380_SixPFJet32_DoublePFBTagDeepCSV_2p2", "HLT_PFHT380_SixPFJet32", "HLT_PFHT430_SixPFJet40_PFBTagCSV_1p5", "HLT_PFHT430_SixPFJet40", "HLT_PFHT350", "HLT_PFHT350MinPFJet15", "HLT_Photon60_R9Id90_CaloIdL_IsoL", "HLT_Photon60_R9Id90_CaloIdL_IsoL_DisplacedIdL", "HLT_Photon60_R9Id90_CaloIdL_IsoL_DisplacedIdL_PFHT350MinPFJet15", "HLT_FullTrack_Multiplicity85", "HLT_FullTrack_Multiplicity100", "HLT_FullTrack_Multiplicity130", "HLT_FullTrack_Multiplicity155", "HLT_HISinglePhoton10_Eta3p1ForPPRef", "HLT_HISinglePhoton20_Eta3p1ForPPRef", "HLT_HISinglePhoton30_Eta3p1ForPPRef", "HLT_HISinglePhoton40_Eta3p1ForPPRef", "HLT_HISinglePhoton50_Eta3p1ForPPRef", "HLT_HISinglePhoton60_Eta3p1ForPPRef", "HLT_Photon20_HoverELoose", "HLT_Photon30_HoverELoose", "HLT_Photon40_HoverELoose", "HLT_Photon50_HoverELoose", "HLT_Photon60_HoverELoose", "HLT_IsoTrackHB", "HLT_IsoTrackHE", "HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTau30_eta2p1_CrossL1", "HLT_Ele24_eta2p1_WPTight_Gsf_MediumChargedIsoPFTau30_eta2p1_CrossL1", "HLT_Ele24_eta2p1_WPTight_Gsf_TightChargedIsoPFTau30_eta2p1_CrossL1", "HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTau30_eta2p1_TightID_CrossL1", "HLT_Ele24_eta2p1_WPTight_Gsf_MediumChargedIsoPFTau30_eta2p1_TightID_CrossL1", "HLT_Ele24_eta2p1_WPTight_Gsf_TightChargedIsoPFTau30_eta2p1_TightID_CrossL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau50_Trk30_eta2p1_1pr", "HLT_IsoMu24_eta2p1_LooseChargedIsoPFTau35_Trk1_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_LooseChargedIsoPFTau35_Trk1_TightID_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau35_Trk1_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau35_Trk1_TightID_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau35_Trk1_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau35_Trk1_TightID_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau40_Trk1_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau40_Trk1_TightID_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau40_Trk1_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau40_Trk1_TightID_eta2p1_Reg_CrossL1", "HLT_Ele16_Ele12_Ele8_CaloIdL_TrackIdL", "HLT_IsoMu27_LooseChargedIsoPFTau20_SingleL1", "HLT_IsoMu27_MediumChargedIsoPFTau20_SingleL1", "HLT_IsoMu27_TightChargedIsoPFTau20_SingleL1", "HLT_Photon50_R9Id90_HE10_IsoM_EBOnly_PFJetsMJJ300DEta3_PFMET50", "HLT_Photon75_R9Id90_HE10_IsoM_EBOnly_PFJetsMJJ300DEta3", "HLT_Photon75_R9Id90_HE10_IsoM_EBOnly_PFJetsMJJ600DEta3", "HLT_Mu18_Mu9_SameSign", "HLT_Mu18_Mu9_SameSign_DZ", "HLT_Mu18_Mu9", "HLT_Mu18_Mu9_DZ", "HLT_Mu20_Mu10_SameSign", "HLT_Mu20_Mu10_SameSign_DZ", "HLT_Mu20_Mu10", "HLT_Mu20_Mu10_DZ", "HLT_Mu23_Mu12_SameSign", "HLT_Mu23_Mu12_SameSign_DZ", "HLT_Mu23_Mu12", "HLT_Mu23_Mu12_DZ", "HLT_TripleMu_5_3_3_Mass3p8to60_DCA", "HLTriggerFinalPath", "L1simulation_step", "Sample", "HT_eventHT", "Muon_effSF", "Electron_effSF", "Muon_effSF_errUp", "Electron_effSF_errUp", "Muon_effSF_errDown", "Electron_effSF_errDown", "puWeight", "puWeightUp", "puWeightDown", "Jet_btagSF_deepjet_M_down", "Jet_btagSF_deepjet_M", "Jet_btagSF_deepjet_M_up", "PrefireWeight", "PrefireWeight_Up", "PrefireWeight_Down", "Jet_pt_raw", "Jet_pt_nom", "Jet_mass_raw", "Jet_mass_nom", "Jet_corr_JEC", "Jet_corr_JER", "MET_T1_pt", "MET_T1_phi", "MET_T1Smear_pt", "MET_T1Smear_phi", "Jet_pt_jerUp", "Jet_mass_jerUp", "MET_T1_pt_jerUp", "MET_T1_phi_jerUp", "MET_T1Smear_pt_jerUp", "MET_T1Smear_phi_jerUp", "Jet_pt_jesAbsoluteStatUp", "Jet_mass_jesAbsoluteStatUp", "MET_T1_pt_jesAbsoluteStatUp", "MET_T1_phi_jesAbsoluteStatUp", "MET_T1Smear_pt_jesAbsoluteStatUp", "MET_T1Smear_phi_jesAbsoluteStatUp", "Jet_pt_jesAbsoluteScaleUp", "Jet_mass_jesAbsoluteScaleUp", "MET_T1_pt_jesAbsoluteScaleUp", "MET_T1_phi_jesAbsoluteScaleUp", "MET_T1Smear_pt_jesAbsoluteScaleUp", "MET_T1Smear_phi_jesAbsoluteScaleUp", "Jet_pt_jesAbsoluteFlavMapUp", "Jet_mass_jesAbsoluteFlavMapUp", "MET_T1_pt_jesAbsoluteFlavMapUp", "MET_T1_phi_jesAbsoluteFlavMapUp", "MET_T1Smear_pt_jesAbsoluteFlavMapUp", "MET_T1Smear_phi_jesAbsoluteFlavMapUp", "Jet_pt_jesAbsoluteMPFBiasUp", "Jet_mass_jesAbsoluteMPFBiasUp", "MET_T1_pt_jesAbsoluteMPFBiasUp", "MET_T1_phi_jesAbsoluteMPFBiasUp", "MET_T1Smear_pt_jesAbsoluteMPFBiasUp", "MET_T1Smear_phi_jesAbsoluteMPFBiasUp", "Jet_pt_jesFragmentationUp", "Jet_mass_jesFragmentationUp", "MET_T1_pt_jesFragmentationUp", "MET_T1_phi_jesFragmentationUp", "MET_T1Smear_pt_jesFragmentationUp", "MET_T1Smear_phi_jesFragmentationUp", "Jet_pt_jesSinglePionECALUp", "Jet_mass_jesSinglePionECALUp", "MET_T1_pt_jesSinglePionECALUp", "MET_T1_phi_jesSinglePionECALUp", "MET_T1Smear_pt_jesSinglePionECALUp", "MET_T1Smear_phi_jesSinglePionECALUp", "Jet_pt_jesSinglePionHCALUp", "Jet_mass_jesSinglePionHCALUp", "MET_T1_pt_jesSinglePionHCALUp", "MET_T1_phi_jesSinglePionHCALUp", "MET_T1Smear_pt_jesSinglePionHCALUp", "MET_T1Smear_phi_jesSinglePionHCALUp", "Jet_pt_jesFlavorQCDUp", "Jet_mass_jesFlavorQCDUp", "MET_T1_pt_jesFlavorQCDUp", "MET_T1_phi_jesFlavorQCDUp", "MET_T1Smear_pt_jesFlavorQCDUp", "MET_T1Smear_phi_jesFlavorQCDUp", "Jet_pt_jesTimePtEtaUp", "Jet_mass_jesTimePtEtaUp", "MET_T1_pt_jesTimePtEtaUp", "MET_T1_phi_jesTimePtEtaUp", "MET_T1Smear_pt_jesTimePtEtaUp", "MET_T1Smear_phi_jesTimePtEtaUp", "Jet_pt_jesRelativeJEREC1Up", "Jet_mass_jesRelativeJEREC1Up", "MET_T1_pt_jesRelativeJEREC1Up", "MET_T1_phi_jesRelativeJEREC1Up", "MET_T1Smear_pt_jesRelativeJEREC1Up", "MET_T1Smear_phi_jesRelativeJEREC1Up", "Jet_pt_jesRelativeJEREC2Up", "Jet_mass_jesRelativeJEREC2Up", "MET_T1_pt_jesRelativeJEREC2Up", "MET_T1_phi_jesRelativeJEREC2Up", "MET_T1Smear_pt_jesRelativeJEREC2Up", "MET_T1Smear_phi_jesRelativeJEREC2Up", "Jet_pt_jesRelativeJERHFUp", "Jet_mass_jesRelativeJERHFUp", "MET_T1_pt_jesRelativeJERHFUp", "MET_T1_phi_jesRelativeJERHFUp", "MET_T1Smear_pt_jesRelativeJERHFUp", "MET_T1Smear_phi_jesRelativeJERHFUp", "Jet_pt_jesRelativePtBBUp", "Jet_mass_jesRelativePtBBUp", "MET_T1_pt_jesRelativePtBBUp", "MET_T1_phi_jesRelativePtBBUp", "MET_T1Smear_pt_jesRelativePtBBUp", "MET_T1Smear_phi_jesRelativePtBBUp", "Jet_pt_jesRelativePtEC1Up", "Jet_mass_jesRelativePtEC1Up", "MET_T1_pt_jesRelativePtEC1Up", "MET_T1_phi_jesRelativePtEC1Up", "MET_T1Smear_pt_jesRelativePtEC1Up", "MET_T1Smear_phi_jesRelativePtEC1Up", "Jet_pt_jesRelativePtEC2Up", "Jet_mass_jesRelativePtEC2Up", "MET_T1_pt_jesRelativePtEC2Up", "MET_T1_phi_jesRelativePtEC2Up", "MET_T1Smear_pt_jesRelativePtEC2Up", "MET_T1Smear_phi_jesRelativePtEC2Up", "Jet_pt_jesRelativePtHFUp", "Jet_mass_jesRelativePtHFUp", "MET_T1_pt_jesRelativePtHFUp", "MET_T1_phi_jesRelativePtHFUp", "MET_T1Smear_pt_jesRelativePtHFUp", "MET_T1Smear_phi_jesRelativePtHFUp", "Jet_pt_jesRelativeBalUp", "Jet_mass_jesRelativeBalUp", "MET_T1_pt_jesRelativeBalUp", "MET_T1_phi_jesRelativeBalUp", "MET_T1Smear_pt_jesRelativeBalUp", "MET_T1Smear_phi_jesRelativeBalUp", "Jet_pt_jesRelativeSampleUp", "Jet_mass_jesRelativeSampleUp", "MET_T1_pt_jesRelativeSampleUp", "MET_T1_phi_jesRelativeSampleUp", "MET_T1Smear_pt_jesRelativeSampleUp", "MET_T1Smear_phi_jesRelativeSampleUp", "Jet_pt_jesRelativeFSRUp", "Jet_mass_jesRelativeFSRUp", "MET_T1_pt_jesRelativeFSRUp", "MET_T1_phi_jesRelativeFSRUp", "MET_T1Smear_pt_jesRelativeFSRUp", "MET_T1Smear_phi_jesRelativeFSRUp", "Jet_pt_jesRelativeStatFSRUp", "Jet_mass_jesRelativeStatFSRUp", "MET_T1_pt_jesRelativeStatFSRUp", "MET_T1_phi_jesRelativeStatFSRUp", "MET_T1Smear_pt_jesRelativeStatFSRUp", "MET_T1Smear_phi_jesRelativeStatFSRUp", "Jet_pt_jesRelativeStatECUp", "Jet_mass_jesRelativeStatECUp", "MET_T1_pt_jesRelativeStatECUp", "MET_T1_phi_jesRelativeStatECUp", "MET_T1Smear_pt_jesRelativeStatECUp", "MET_T1Smear_phi_jesRelativeStatECUp", "Jet_pt_jesRelativeStatHFUp", "Jet_mass_jesRelativeStatHFUp", "MET_T1_pt_jesRelativeStatHFUp", "MET_T1_phi_jesRelativeStatHFUp", "MET_T1Smear_pt_jesRelativeStatHFUp", "MET_T1Smear_phi_jesRelativeStatHFUp", "Jet_pt_jesPileUpDataMCUp", "Jet_mass_jesPileUpDataMCUp", "MET_T1_pt_jesPileUpDataMCUp", "MET_T1_phi_jesPileUpDataMCUp", "MET_T1Smear_pt_jesPileUpDataMCUp", "MET_T1Smear_phi_jesPileUpDataMCUp", "Jet_pt_jesPileUpPtRefUp", "Jet_mass_jesPileUpPtRefUp", "MET_T1_pt_jesPileUpPtRefUp", "MET_T1_phi_jesPileUpPtRefUp", "MET_T1Smear_pt_jesPileUpPtRefUp", "MET_T1Smear_phi_jesPileUpPtRefUp", "Jet_pt_jesPileUpPtBBUp", "Jet_mass_jesPileUpPtBBUp", "MET_T1_pt_jesPileUpPtBBUp", "MET_T1_phi_jesPileUpPtBBUp", "MET_T1Smear_pt_jesPileUpPtBBUp", "MET_T1Smear_phi_jesPileUpPtBBUp", "Jet_pt_jesPileUpPtEC1Up", "Jet_mass_jesPileUpPtEC1Up", "MET_T1_pt_jesPileUpPtEC1Up", "MET_T1_phi_jesPileUpPtEC1Up", "MET_T1Smear_pt_jesPileUpPtEC1Up", "MET_T1Smear_phi_jesPileUpPtEC1Up", "Jet_pt_jesPileUpPtEC2Up", "Jet_mass_jesPileUpPtEC2Up", "MET_T1_pt_jesPileUpPtEC2Up", "MET_T1_phi_jesPileUpPtEC2Up", "MET_T1Smear_pt_jesPileUpPtEC2Up", "MET_T1Smear_phi_jesPileUpPtEC2Up", "Jet_pt_jesPileUpPtHFUp", "Jet_mass_jesPileUpPtHFUp", "MET_T1_pt_jesPileUpPtHFUp", "MET_T1_phi_jesPileUpPtHFUp", "MET_T1Smear_pt_jesPileUpPtHFUp", "MET_T1Smear_phi_jesPileUpPtHFUp", "Jet_pt_jesPileUpMuZeroUp", "Jet_mass_jesPileUpMuZeroUp", "MET_T1_pt_jesPileUpMuZeroUp", "MET_T1_phi_jesPileUpMuZeroUp", "MET_T1Smear_pt_jesPileUpMuZeroUp", "MET_T1Smear_phi_jesPileUpMuZeroUp", "Jet_pt_jesPileUpEnvelopeUp", "Jet_mass_jesPileUpEnvelopeUp", "MET_T1_pt_jesPileUpEnvelopeUp", "MET_T1_phi_jesPileUpEnvelopeUp", "MET_T1Smear_pt_jesPileUpEnvelopeUp", "MET_T1Smear_phi_jesPileUpEnvelopeUp", "Jet_pt_jesSubTotalPileUpUp", "Jet_mass_jesSubTotalPileUpUp", "MET_T1_pt_jesSubTotalPileUpUp", "MET_T1_phi_jesSubTotalPileUpUp", "MET_T1Smear_pt_jesSubTotalPileUpUp", "MET_T1Smear_phi_jesSubTotalPileUpUp", "Jet_pt_jesSubTotalRelativeUp", "Jet_mass_jesSubTotalRelativeUp", "MET_T1_pt_jesSubTotalRelativeUp", "MET_T1_phi_jesSubTotalRelativeUp", "MET_T1Smear_pt_jesSubTotalRelativeUp", "MET_T1Smear_phi_jesSubTotalRelativeUp", "Jet_pt_jesSubTotalPtUp", "Jet_mass_jesSubTotalPtUp", "MET_T1_pt_jesSubTotalPtUp", "MET_T1_phi_jesSubTotalPtUp", "MET_T1Smear_pt_jesSubTotalPtUp", "MET_T1Smear_phi_jesSubTotalPtUp", "Jet_pt_jesSubTotalScaleUp", "Jet_mass_jesSubTotalScaleUp", "MET_T1_pt_jesSubTotalScaleUp", "MET_T1_phi_jesSubTotalScaleUp", "MET_T1Smear_pt_jesSubTotalScaleUp", "MET_T1Smear_phi_jesSubTotalScaleUp", "Jet_pt_jesSubTotalAbsoluteUp", "Jet_mass_jesSubTotalAbsoluteUp", "MET_T1_pt_jesSubTotalAbsoluteUp", "MET_T1_phi_jesSubTotalAbsoluteUp", "MET_T1Smear_pt_jesSubTotalAbsoluteUp", "MET_T1Smear_phi_jesSubTotalAbsoluteUp", "Jet_pt_jesSubTotalMCUp", "Jet_mass_jesSubTotalMCUp", "MET_T1_pt_jesSubTotalMCUp", "MET_T1_phi_jesSubTotalMCUp", "MET_T1Smear_pt_jesSubTotalMCUp", "MET_T1Smear_phi_jesSubTotalMCUp", "Jet_pt_jesTotalUp", "Jet_mass_jesTotalUp", "MET_T1_pt_jesTotalUp", "MET_T1_phi_jesTotalUp", "MET_T1Smear_pt_jesTotalUp", "MET_T1Smear_phi_jesTotalUp", "Jet_pt_jesTotalNoFlavorUp", "Jet_mass_jesTotalNoFlavorUp", "MET_T1_pt_jesTotalNoFlavorUp", "MET_T1_phi_jesTotalNoFlavorUp", "MET_T1Smear_pt_jesTotalNoFlavorUp", "MET_T1Smear_phi_jesTotalNoFlavorUp", "Jet_pt_jesTotalNoTimeUp", "Jet_mass_jesTotalNoTimeUp", "MET_T1_pt_jesTotalNoTimeUp", "MET_T1_phi_jesTotalNoTimeUp", "MET_T1Smear_pt_jesTotalNoTimeUp", "MET_T1Smear_phi_jesTotalNoTimeUp", "Jet_pt_jesTotalNoFlavorNoTimeUp", "Jet_mass_jesTotalNoFlavorNoTimeUp", "MET_T1_pt_jesTotalNoFlavorNoTimeUp", "MET_T1_phi_jesTotalNoFlavorNoTimeUp", "MET_T1Smear_pt_jesTotalNoFlavorNoTimeUp", "MET_T1Smear_phi_jesTotalNoFlavorNoTimeUp", "Jet_pt_jesFlavorZJetUp", "Jet_mass_jesFlavorZJetUp", "MET_T1_pt_jesFlavorZJetUp", "MET_T1_phi_jesFlavorZJetUp", "MET_T1Smear_pt_jesFlavorZJetUp", "MET_T1Smear_phi_jesFlavorZJetUp", "Jet_pt_jesFlavorPhotonJetUp", "Jet_mass_jesFlavorPhotonJetUp", "MET_T1_pt_jesFlavorPhotonJetUp", "MET_T1_phi_jesFlavorPhotonJetUp", "MET_T1Smear_pt_jesFlavorPhotonJetUp", "MET_T1Smear_phi_jesFlavorPhotonJetUp", "Jet_pt_jesFlavorPureGluonUp", "Jet_mass_jesFlavorPureGluonUp", "MET_T1_pt_jesFlavorPureGluonUp", "MET_T1_phi_jesFlavorPureGluonUp", "MET_T1Smear_pt_jesFlavorPureGluonUp", "MET_T1Smear_phi_jesFlavorPureGluonUp", "Jet_pt_jesFlavorPureQuarkUp", "Jet_mass_jesFlavorPureQuarkUp", "MET_T1_pt_jesFlavorPureQuarkUp", "MET_T1_phi_jesFlavorPureQuarkUp", "MET_T1Smear_pt_jesFlavorPureQuarkUp", "MET_T1Smear_phi_jesFlavorPureQuarkUp", "Jet_pt_jesFlavorPureCharmUp", "Jet_mass_jesFlavorPureCharmUp", "MET_T1_pt_jesFlavorPureCharmUp", "MET_T1_phi_jesFlavorPureCharmUp", "MET_T1Smear_pt_jesFlavorPureCharmUp", "MET_T1Smear_phi_jesFlavorPureCharmUp", "Jet_pt_jesFlavorPureBottomUp", "Jet_mass_jesFlavorPureBottomUp", "MET_T1_pt_jesFlavorPureBottomUp", "MET_T1_phi_jesFlavorPureBottomUp", "MET_T1Smear_pt_jesFlavorPureBottomUp", "MET_T1Smear_phi_jesFlavorPureBottomUp", "Jet_pt_jesTimeRunBUp", "Jet_mass_jesTimeRunBUp", "MET_T1_pt_jesTimeRunBUp", "MET_T1_phi_jesTimeRunBUp", "MET_T1Smear_pt_jesTimeRunBUp", "MET_T1Smear_phi_jesTimeRunBUp", "Jet_pt_jesTimeRunCUp", "Jet_mass_jesTimeRunCUp", "MET_T1_pt_jesTimeRunCUp", "MET_T1_phi_jesTimeRunCUp", "MET_T1Smear_pt_jesTimeRunCUp", "MET_T1Smear_phi_jesTimeRunCUp", "Jet_pt_jesTimeRunDEUp", "Jet_mass_jesTimeRunDEUp", "MET_T1_pt_jesTimeRunDEUp", "MET_T1_phi_jesTimeRunDEUp", "MET_T1Smear_pt_jesTimeRunDEUp", "MET_T1Smear_phi_jesTimeRunDEUp", "Jet_pt_jesTimeRunFUp", "Jet_mass_jesTimeRunFUp", "MET_T1_pt_jesTimeRunFUp", "MET_T1_phi_jesTimeRunFUp", "MET_T1Smear_pt_jesTimeRunFUp", "MET_T1Smear_phi_jesTimeRunFUp", "Jet_pt_jesCorrelationGroupMPFInSituUp", "Jet_mass_jesCorrelationGroupMPFInSituUp", "MET_T1_pt_jesCorrelationGroupMPFInSituUp", "MET_T1_phi_jesCorrelationGroupMPFInSituUp", "MET_T1Smear_pt_jesCorrelationGroupMPFInSituUp", "MET_T1Smear_phi_jesCorrelationGroupMPFInSituUp", "Jet_pt_jesCorrelationGroupIntercalibrationUp", "Jet_mass_jesCorrelationGroupIntercalibrationUp", "MET_T1_pt_jesCorrelationGroupIntercalibrationUp", "MET_T1_phi_jesCorrelationGroupIntercalibrationUp", "MET_T1Smear_pt_jesCorrelationGroupIntercalibrationUp", "MET_T1Smear_phi_jesCorrelationGroupIntercalibrationUp", "Jet_pt_jesCorrelationGroupbJESUp", "Jet_mass_jesCorrelationGroupbJESUp", "MET_T1_pt_jesCorrelationGroupbJESUp", "MET_T1_phi_jesCorrelationGroupbJESUp", "MET_T1Smear_pt_jesCorrelationGroupbJESUp", "MET_T1Smear_phi_jesCorrelationGroupbJESUp", "Jet_pt_jesCorrelationGroupFlavorUp", "Jet_mass_jesCorrelationGroupFlavorUp", "MET_T1_pt_jesCorrelationGroupFlavorUp", "MET_T1_phi_jesCorrelationGroupFlavorUp", "MET_T1Smear_pt_jesCorrelationGroupFlavorUp", "MET_T1Smear_phi_jesCorrelationGroupFlavorUp", "Jet_pt_jesCorrelationGroupUncorrelatedUp", "Jet_mass_jesCorrelationGroupUncorrelatedUp", "MET_T1_pt_jesCorrelationGroupUncorrelatedUp", "MET_T1_phi_jesCorrelationGroupUncorrelatedUp", "MET_T1Smear_pt_jesCorrelationGroupUncorrelatedUp", "MET_T1Smear_phi_jesCorrelationGroupUncorrelatedUp", "MET_T1_pt_unclustEnUp", "MET_T1_phi_unclustEnUp", "MET_T1Smear_pt_unclustEnUp", "MET_T1Smear_phi_unclustEnUp", "Jet_pt_jerDown", "Jet_mass_jerDown", "MET_T1_pt_jerDown", "MET_T1_phi_jerDown", "MET_T1Smear_pt_jerDown", "MET_T1Smear_phi_jerDown", "Jet_pt_jesAbsoluteStatDown", "Jet_mass_jesAbsoluteStatDown", "MET_T1_pt_jesAbsoluteStatDown", "MET_T1_phi_jesAbsoluteStatDown", "MET_T1Smear_pt_jesAbsoluteStatDown", "MET_T1Smear_phi_jesAbsoluteStatDown", "Jet_pt_jesAbsoluteScaleDown", "Jet_mass_jesAbsoluteScaleDown", "MET_T1_pt_jesAbsoluteScaleDown", "MET_T1_phi_jesAbsoluteScaleDown", "MET_T1Smear_pt_jesAbsoluteScaleDown", "MET_T1Smear_phi_jesAbsoluteScaleDown", "Jet_pt_jesAbsoluteFlavMapDown", "Jet_mass_jesAbsoluteFlavMapDown", "MET_T1_pt_jesAbsoluteFlavMapDown", "MET_T1_phi_jesAbsoluteFlavMapDown", "MET_T1Smear_pt_jesAbsoluteFlavMapDown", "MET_T1Smear_phi_jesAbsoluteFlavMapDown", "Jet_pt_jesAbsoluteMPFBiasDown", "Jet_mass_jesAbsoluteMPFBiasDown", "MET_T1_pt_jesAbsoluteMPFBiasDown", "MET_T1_phi_jesAbsoluteMPFBiasDown", "MET_T1Smear_pt_jesAbsoluteMPFBiasDown", "MET_T1Smear_phi_jesAbsoluteMPFBiasDown", "Jet_pt_jesFragmentationDown", "Jet_mass_jesFragmentationDown", "MET_T1_pt_jesFragmentationDown", "MET_T1_phi_jesFragmentationDown", "MET_T1Smear_pt_jesFragmentationDown", "MET_T1Smear_phi_jesFragmentationDown", "Jet_pt_jesSinglePionECALDown", "Jet_mass_jesSinglePionECALDown", "MET_T1_pt_jesSinglePionECALDown", "MET_T1_phi_jesSinglePionECALDown", "MET_T1Smear_pt_jesSinglePionECALDown", "MET_T1Smear_phi_jesSinglePionECALDown", "Jet_pt_jesSinglePionHCALDown", "Jet_mass_jesSinglePionHCALDown", "MET_T1_pt_jesSinglePionHCALDown", "MET_T1_phi_jesSinglePionHCALDown", "MET_T1Smear_pt_jesSinglePionHCALDown", "MET_T1Smear_phi_jesSinglePionHCALDown", "Jet_pt_jesFlavorQCDDown", "Jet_mass_jesFlavorQCDDown", "MET_T1_pt_jesFlavorQCDDown", "MET_T1_phi_jesFlavorQCDDown", "MET_T1Smear_pt_jesFlavorQCDDown", "MET_T1Smear_phi_jesFlavorQCDDown", "Jet_pt_jesTimePtEtaDown", "Jet_mass_jesTimePtEtaDown", "MET_T1_pt_jesTimePtEtaDown", "MET_T1_phi_jesTimePtEtaDown", "MET_T1Smear_pt_jesTimePtEtaDown", "MET_T1Smear_phi_jesTimePtEtaDown", "Jet_pt_jesRelativeJEREC1Down", "Jet_mass_jesRelativeJEREC1Down", "MET_T1_pt_jesRelativeJEREC1Down", "MET_T1_phi_jesRelativeJEREC1Down", "MET_T1Smear_pt_jesRelativeJEREC1Down", "MET_T1Smear_phi_jesRelativeJEREC1Down", "Jet_pt_jesRelativeJEREC2Down", "Jet_mass_jesRelativeJEREC2Down", "MET_T1_pt_jesRelativeJEREC2Down", "MET_T1_phi_jesRelativeJEREC2Down", "MET_T1Smear_pt_jesRelativeJEREC2Down", "MET_T1Smear_phi_jesRelativeJEREC2Down", "Jet_pt_jesRelativeJERHFDown", "Jet_mass_jesRelativeJERHFDown", "MET_T1_pt_jesRelativeJERHFDown", "MET_T1_phi_jesRelativeJERHFDown", "MET_T1Smear_pt_jesRelativeJERHFDown", "MET_T1Smear_phi_jesRelativeJERHFDown", "Jet_pt_jesRelativePtBBDown", "Jet_mass_jesRelativePtBBDown", "MET_T1_pt_jesRelativePtBBDown", "MET_T1_phi_jesRelativePtBBDown", "MET_T1Smear_pt_jesRelativePtBBDown", "MET_T1Smear_phi_jesRelativePtBBDown", "Jet_pt_jesRelativePtEC1Down", "Jet_mass_jesRelativePtEC1Down", "MET_T1_pt_jesRelativePtEC1Down", "MET_T1_phi_jesRelativePtEC1Down", "MET_T1Smear_pt_jesRelativePtEC1Down", "MET_T1Smear_phi_jesRelativePtEC1Down", "Jet_pt_jesRelativePtEC2Down", "Jet_mass_jesRelativePtEC2Down", "MET_T1_pt_jesRelativePtEC2Down", "MET_T1_phi_jesRelativePtEC2Down", "MET_T1Smear_pt_jesRelativePtEC2Down", "MET_T1Smear_phi_jesRelativePtEC2Down", "Jet_pt_jesRelativePtHFDown", "Jet_mass_jesRelativePtHFDown", "MET_T1_pt_jesRelativePtHFDown", "MET_T1_phi_jesRelativePtHFDown", "MET_T1Smear_pt_jesRelativePtHFDown", "MET_T1Smear_phi_jesRelativePtHFDown", "Jet_pt_jesRelativeBalDown", "Jet_mass_jesRelativeBalDown", "MET_T1_pt_jesRelativeBalDown", "MET_T1_phi_jesRelativeBalDown", "MET_T1Smear_pt_jesRelativeBalDown", "MET_T1Smear_phi_jesRelativeBalDown", "Jet_pt_jesRelativeSampleDown", "Jet_mass_jesRelativeSampleDown", "MET_T1_pt_jesRelativeSampleDown", "MET_T1_phi_jesRelativeSampleDown", "MET_T1Smear_pt_jesRelativeSampleDown", "MET_T1Smear_phi_jesRelativeSampleDown", "Jet_pt_jesRelativeFSRDown", "Jet_mass_jesRelativeFSRDown", "MET_T1_pt_jesRelativeFSRDown", "MET_T1_phi_jesRelativeFSRDown", "MET_T1Smear_pt_jesRelativeFSRDown", "MET_T1Smear_phi_jesRelativeFSRDown", "Jet_pt_jesRelativeStatFSRDown", "Jet_mass_jesRelativeStatFSRDown", "MET_T1_pt_jesRelativeStatFSRDown", "MET_T1_phi_jesRelativeStatFSRDown", "MET_T1Smear_pt_jesRelativeStatFSRDown", "MET_T1Smear_phi_jesRelativeStatFSRDown", "Jet_pt_jesRelativeStatECDown", "Jet_mass_jesRelativeStatECDown", "MET_T1_pt_jesRelativeStatECDown", "MET_T1_phi_jesRelativeStatECDown", "MET_T1Smear_pt_jesRelativeStatECDown", "MET_T1Smear_phi_jesRelativeStatECDown", "Jet_pt_jesRelativeStatHFDown", "Jet_mass_jesRelativeStatHFDown", "MET_T1_pt_jesRelativeStatHFDown", "MET_T1_phi_jesRelativeStatHFDown", "MET_T1Smear_pt_jesRelativeStatHFDown", "MET_T1Smear_phi_jesRelativeStatHFDown", "Jet_pt_jesPileUpDataMCDown", "Jet_mass_jesPileUpDataMCDown", "MET_T1_pt_jesPileUpDataMCDown", "MET_T1_phi_jesPileUpDataMCDown", "MET_T1Smear_pt_jesPileUpDataMCDown", "MET_T1Smear_phi_jesPileUpDataMCDown", "Jet_pt_jesPileUpPtRefDown", "Jet_mass_jesPileUpPtRefDown", "MET_T1_pt_jesPileUpPtRefDown", "MET_T1_phi_jesPileUpPtRefDown", "MET_T1Smear_pt_jesPileUpPtRefDown", "MET_T1Smear_phi_jesPileUpPtRefDown", "Jet_pt_jesPileUpPtBBDown", "Jet_mass_jesPileUpPtBBDown", "MET_T1_pt_jesPileUpPtBBDown", "MET_T1_phi_jesPileUpPtBBDown", "MET_T1Smear_pt_jesPileUpPtBBDown", "MET_T1Smear_phi_jesPileUpPtBBDown", "Jet_pt_jesPileUpPtEC1Down", "Jet_mass_jesPileUpPtEC1Down", "MET_T1_pt_jesPileUpPtEC1Down", "MET_T1_phi_jesPileUpPtEC1Down", "MET_T1Smear_pt_jesPileUpPtEC1Down", "MET_T1Smear_phi_jesPileUpPtEC1Down", "Jet_pt_jesPileUpPtEC2Down", "Jet_mass_jesPileUpPtEC2Down", "MET_T1_pt_jesPileUpPtEC2Down", "MET_T1_phi_jesPileUpPtEC2Down", "MET_T1Smear_pt_jesPileUpPtEC2Down", "MET_T1Smear_phi_jesPileUpPtEC2Down", "Jet_pt_jesPileUpPtHFDown", "Jet_mass_jesPileUpPtHFDown", "MET_T1_pt_jesPileUpPtHFDown", "MET_T1_phi_jesPileUpPtHFDown", "MET_T1Smear_pt_jesPileUpPtHFDown", "MET_T1Smear_phi_jesPileUpPtHFDown", "Jet_pt_jesPileUpMuZeroDown", "Jet_mass_jesPileUpMuZeroDown", "MET_T1_pt_jesPileUpMuZeroDown", "MET_T1_phi_jesPileUpMuZeroDown", "MET_T1Smear_pt_jesPileUpMuZeroDown", "MET_T1Smear_phi_jesPileUpMuZeroDown", "Jet_pt_jesPileUpEnvelopeDown", "Jet_mass_jesPileUpEnvelopeDown", "MET_T1_pt_jesPileUpEnvelopeDown", "MET_T1_phi_jesPileUpEnvelopeDown", "MET_T1Smear_pt_jesPileUpEnvelopeDown", "MET_T1Smear_phi_jesPileUpEnvelopeDown", "Jet_pt_jesSubTotalPileUpDown", "Jet_mass_jesSubTotalPileUpDown", "MET_T1_pt_jesSubTotalPileUpDown", "MET_T1_phi_jesSubTotalPileUpDown", "MET_T1Smear_pt_jesSubTotalPileUpDown", "MET_T1Smear_phi_jesSubTotalPileUpDown", "Jet_pt_jesSubTotalRelativeDown", "Jet_mass_jesSubTotalRelativeDown", "MET_T1_pt_jesSubTotalRelativeDown", "MET_T1_phi_jesSubTotalRelativeDown", "MET_T1Smear_pt_jesSubTotalRelativeDown", "MET_T1Smear_phi_jesSubTotalRelativeDown", "Jet_pt_jesSubTotalPtDown", "Jet_mass_jesSubTotalPtDown", "MET_T1_pt_jesSubTotalPtDown", "MET_T1_phi_jesSubTotalPtDown", "MET_T1Smear_pt_jesSubTotalPtDown", "MET_T1Smear_phi_jesSubTotalPtDown", "Jet_pt_jesSubTotalScaleDown", "Jet_mass_jesSubTotalScaleDown", "MET_T1_pt_jesSubTotalScaleDown", "MET_T1_phi_jesSubTotalScaleDown", "MET_T1Smear_pt_jesSubTotalScaleDown", "MET_T1Smear_phi_jesSubTotalScaleDown", "Jet_pt_jesSubTotalAbsoluteDown", "Jet_mass_jesSubTotalAbsoluteDown", "MET_T1_pt_jesSubTotalAbsoluteDown", "MET_T1_phi_jesSubTotalAbsoluteDown", "MET_T1Smear_pt_jesSubTotalAbsoluteDown", "MET_T1Smear_phi_jesSubTotalAbsoluteDown", "Jet_pt_jesSubTotalMCDown", "Jet_mass_jesSubTotalMCDown", "MET_T1_pt_jesSubTotalMCDown", "MET_T1_phi_jesSubTotalMCDown", "MET_T1Smear_pt_jesSubTotalMCDown", "MET_T1Smear_phi_jesSubTotalMCDown", "Jet_pt_jesTotalDown", "Jet_mass_jesTotalDown", "MET_T1_pt_jesTotalDown", "MET_T1_phi_jesTotalDown", "MET_T1Smear_pt_jesTotalDown", "MET_T1Smear_phi_jesTotalDown", "Jet_pt_jesTotalNoFlavorDown", "Jet_mass_jesTotalNoFlavorDown", "MET_T1_pt_jesTotalNoFlavorDown", "MET_T1_phi_jesTotalNoFlavorDown", "MET_T1Smear_pt_jesTotalNoFlavorDown", "MET_T1Smear_phi_jesTotalNoFlavorDown", "Jet_pt_jesTotalNoTimeDown", "Jet_mass_jesTotalNoTimeDown", "MET_T1_pt_jesTotalNoTimeDown", "MET_T1_phi_jesTotalNoTimeDown", "MET_T1Smear_pt_jesTotalNoTimeDown", "MET_T1Smear_phi_jesTotalNoTimeDown", "Jet_pt_jesTotalNoFlavorNoTimeDown", "Jet_mass_jesTotalNoFlavorNoTimeDown", "MET_T1_pt_jesTotalNoFlavorNoTimeDown", "MET_T1_phi_jesTotalNoFlavorNoTimeDown", "MET_T1Smear_pt_jesTotalNoFlavorNoTimeDown", "MET_T1Smear_phi_jesTotalNoFlavorNoTimeDown", "Jet_pt_jesFlavorZJetDown", "Jet_mass_jesFlavorZJetDown", "MET_T1_pt_jesFlavorZJetDown", "MET_T1_phi_jesFlavorZJetDown", "MET_T1Smear_pt_jesFlavorZJetDown", "MET_T1Smear_phi_jesFlavorZJetDown", "Jet_pt_jesFlavorPhotonJetDown", "Jet_mass_jesFlavorPhotonJetDown", "MET_T1_pt_jesFlavorPhotonJetDown", "MET_T1_phi_jesFlavorPhotonJetDown", "MET_T1Smear_pt_jesFlavorPhotonJetDown", "MET_T1Smear_phi_jesFlavorPhotonJetDown", "Jet_pt_jesFlavorPureGluonDown", "Jet_mass_jesFlavorPureGluonDown", "MET_T1_pt_jesFlavorPureGluonDown", "MET_T1_phi_jesFlavorPureGluonDown", "MET_T1Smear_pt_jesFlavorPureGluonDown", "MET_T1Smear_phi_jesFlavorPureGluonDown", "Jet_pt_jesFlavorPureQuarkDown", "Jet_mass_jesFlavorPureQuarkDown", "MET_T1_pt_jesFlavorPureQuarkDown", "MET_T1_phi_jesFlavorPureQuarkDown", "MET_T1Smear_pt_jesFlavorPureQuarkDown", "MET_T1Smear_phi_jesFlavorPureQuarkDown", "Jet_pt_jesFlavorPureCharmDown", "Jet_mass_jesFlavorPureCharmDown", "MET_T1_pt_jesFlavorPureCharmDown", "MET_T1_phi_jesFlavorPureCharmDown", "MET_T1Smear_pt_jesFlavorPureCharmDown", "MET_T1Smear_phi_jesFlavorPureCharmDown", "Jet_pt_jesFlavorPureBottomDown", "Jet_mass_jesFlavorPureBottomDown", "MET_T1_pt_jesFlavorPureBottomDown", "MET_T1_phi_jesFlavorPureBottomDown", "MET_T1Smear_pt_jesFlavorPureBottomDown", "MET_T1Smear_phi_jesFlavorPureBottomDown", "Jet_pt_jesTimeRunBDown", "Jet_mass_jesTimeRunBDown", "MET_T1_pt_jesTimeRunBDown", "MET_T1_phi_jesTimeRunBDown", "MET_T1Smear_pt_jesTimeRunBDown", "MET_T1Smear_phi_jesTimeRunBDown", "Jet_pt_jesTimeRunCDown", "Jet_mass_jesTimeRunCDown", "MET_T1_pt_jesTimeRunCDown", "MET_T1_phi_jesTimeRunCDown", "MET_T1Smear_pt_jesTimeRunCDown", "MET_T1Smear_phi_jesTimeRunCDown", "Jet_pt_jesTimeRunDEDown", "Jet_mass_jesTimeRunDEDown", "MET_T1_pt_jesTimeRunDEDown", "MET_T1_phi_jesTimeRunDEDown", "MET_T1Smear_pt_jesTimeRunDEDown", "MET_T1Smear_phi_jesTimeRunDEDown", "Jet_pt_jesTimeRunFDown", "Jet_mass_jesTimeRunFDown", "MET_T1_pt_jesTimeRunFDown", "MET_T1_phi_jesTimeRunFDown", "MET_T1Smear_pt_jesTimeRunFDown", "MET_T1Smear_phi_jesTimeRunFDown", "Jet_pt_jesCorrelationGroupMPFInSituDown", "Jet_mass_jesCorrelationGroupMPFInSituDown", "MET_T1_pt_jesCorrelationGroupMPFInSituDown", "MET_T1_phi_jesCorrelationGroupMPFInSituDown", "MET_T1Smear_pt_jesCorrelationGroupMPFInSituDown", "MET_T1Smear_phi_jesCorrelationGroupMPFInSituDown", "Jet_pt_jesCorrelationGroupIntercalibrationDown", "Jet_mass_jesCorrelationGroupIntercalibrationDown", "MET_T1_pt_jesCorrelationGroupIntercalibrationDown", "MET_T1_phi_jesCorrelationGroupIntercalibrationDown", "MET_T1Smear_pt_jesCorrelationGroupIntercalibrationDown", "MET_T1Smear_phi_jesCorrelationGroupIntercalibrationDown", "Jet_pt_jesCorrelationGroupbJESDown", "Jet_mass_jesCorrelationGroupbJESDown", "MET_T1_pt_jesCorrelationGroupbJESDown", "MET_T1_phi_jesCorrelationGroupbJESDown", "MET_T1Smear_pt_jesCorrelationGroupbJESDown", "MET_T1Smear_phi_jesCorrelationGroupbJESDown", "Jet_pt_jesCorrelationGroupFlavorDown", "Jet_mass_jesCorrelationGroupFlavorDown", "MET_T1_pt_jesCorrelationGroupFlavorDown", "MET_T1_phi_jesCorrelationGroupFlavorDown", "MET_T1Smear_pt_jesCorrelationGroupFlavorDown", "MET_T1Smear_phi_jesCorrelationGroupFlavorDown", "Jet_pt_jesCorrelationGroupUncorrelatedDown", "Jet_mass_jesCorrelationGroupUncorrelatedDown", "MET_T1_pt_jesCorrelationGroupUncorrelatedDown", "MET_T1_phi_jesCorrelationGroupUncorrelatedDown", "MET_T1Smear_pt_jesCorrelationGroupUncorrelatedDown", "MET_T1Smear_phi_jesCorrelationGroupUncorrelatedDown", "MET_T1_pt_unclustEnDown", "MET_T1_phi_unclustEnDown", "MET_T1Smear_pt_unclustEnDown", "MET_T1Smear_phi_unclustEnDown", "FatJet_pt_raw", "FatJet_pt_nom", "FatJet_mass_raw", "FatJet_mass_nom", "FatJet_corr_JEC", "FatJet_corr_JER", "FatJet_corr_JMS", "FatJet_corr_JMR", "FatJet_msoftdrop_raw", "FatJet_msoftdrop_nom", "FatJet_msoftdrop_corr_JMR", "FatJet_msoftdrop_corr_JMS", "FatJet_msoftdrop_corr_PUPPI", "FatJet_msoftdrop_tau21DDT_nom", "FatJet_pt_jerUp", "FatJet_mass_jerUp", "FatJet_mass_jmrUp", "FatJet_mass_jmsUp", "FatJet_msoftdrop_jerUp", "FatJet_msoftdrop_tau21DDT_jerUp", "FatJet_msoftdrop_jmrUp", "FatJet_msoftdrop_jmsUp", "FatJet_msoftdrop_tau21DDT_jmrUp", "FatJet_msoftdrop_tau21DDT_jmsUp", "FatJet_pt_jesAbsoluteStatUp", "FatJet_mass_jesAbsoluteStatUp", "FatJet_msoftdrop_jesAbsoluteStatUp", "FatJet_pt_jesAbsoluteScaleUp", "FatJet_mass_jesAbsoluteScaleUp", "FatJet_msoftdrop_jesAbsoluteScaleUp", "FatJet_pt_jesAbsoluteFlavMapUp", "FatJet_mass_jesAbsoluteFlavMapUp", "FatJet_msoftdrop_jesAbsoluteFlavMapUp", "FatJet_pt_jesAbsoluteMPFBiasUp", "FatJet_mass_jesAbsoluteMPFBiasUp", "FatJet_msoftdrop_jesAbsoluteMPFBiasUp", "FatJet_pt_jesFragmentationUp", "FatJet_mass_jesFragmentationUp", "FatJet_msoftdrop_jesFragmentationUp", "FatJet_pt_jesSinglePionECALUp", "FatJet_mass_jesSinglePionECALUp", "FatJet_msoftdrop_jesSinglePionECALUp", "FatJet_pt_jesSinglePionHCALUp", "FatJet_mass_jesSinglePionHCALUp", "FatJet_msoftdrop_jesSinglePionHCALUp", "FatJet_pt_jesFlavorQCDUp", "FatJet_mass_jesFlavorQCDUp", "FatJet_msoftdrop_jesFlavorQCDUp", "FatJet_pt_jesTimePtEtaUp", "FatJet_mass_jesTimePtEtaUp", "FatJet_msoftdrop_jesTimePtEtaUp", "FatJet_pt_jesRelativeJEREC1Up", "FatJet_mass_jesRelativeJEREC1Up", "FatJet_msoftdrop_jesRelativeJEREC1Up", "FatJet_pt_jesRelativeJEREC2Up", "FatJet_mass_jesRelativeJEREC2Up", "FatJet_msoftdrop_jesRelativeJEREC2Up", "FatJet_pt_jesRelativeJERHFUp", "FatJet_mass_jesRelativeJERHFUp", "FatJet_msoftdrop_jesRelativeJERHFUp", "FatJet_pt_jesRelativePtBBUp", "FatJet_mass_jesRelativePtBBUp", "FatJet_msoftdrop_jesRelativePtBBUp", "FatJet_pt_jesRelativePtEC1Up", "FatJet_mass_jesRelativePtEC1Up", "FatJet_msoftdrop_jesRelativePtEC1Up", "FatJet_pt_jesRelativePtEC2Up", "FatJet_mass_jesRelativePtEC2Up", "FatJet_msoftdrop_jesRelativePtEC2Up", "FatJet_pt_jesRelativePtHFUp", "FatJet_mass_jesRelativePtHFUp", "FatJet_msoftdrop_jesRelativePtHFUp", "FatJet_pt_jesRelativeBalUp", "FatJet_mass_jesRelativeBalUp", "FatJet_msoftdrop_jesRelativeBalUp", "FatJet_pt_jesRelativeSampleUp", "FatJet_mass_jesRelativeSampleUp", "FatJet_msoftdrop_jesRelativeSampleUp", "FatJet_pt_jesRelativeFSRUp", "FatJet_mass_jesRelativeFSRUp", "FatJet_msoftdrop_jesRelativeFSRUp", "FatJet_pt_jesRelativeStatFSRUp", "FatJet_mass_jesRelativeStatFSRUp", "FatJet_msoftdrop_jesRelativeStatFSRUp", "FatJet_pt_jesRelativeStatECUp", "FatJet_mass_jesRelativeStatECUp", "FatJet_msoftdrop_jesRelativeStatECUp", "FatJet_pt_jesRelativeStatHFUp", "FatJet_mass_jesRelativeStatHFUp", "FatJet_msoftdrop_jesRelativeStatHFUp", "FatJet_pt_jesPileUpDataMCUp", "FatJet_mass_jesPileUpDataMCUp", "FatJet_msoftdrop_jesPileUpDataMCUp", "FatJet_pt_jesPileUpPtRefUp", "FatJet_mass_jesPileUpPtRefUp", "FatJet_msoftdrop_jesPileUpPtRefUp", "FatJet_pt_jesPileUpPtBBUp", "FatJet_mass_jesPileUpPtBBUp", "FatJet_msoftdrop_jesPileUpPtBBUp", "FatJet_pt_jesPileUpPtEC1Up", "FatJet_mass_jesPileUpPtEC1Up", "FatJet_msoftdrop_jesPileUpPtEC1Up", "FatJet_pt_jesPileUpPtEC2Up", "FatJet_mass_jesPileUpPtEC2Up", "FatJet_msoftdrop_jesPileUpPtEC2Up", "FatJet_pt_jesPileUpPtHFUp", "FatJet_mass_jesPileUpPtHFUp", "FatJet_msoftdrop_jesPileUpPtHFUp", "FatJet_pt_jesPileUpMuZeroUp", "FatJet_mass_jesPileUpMuZeroUp", "FatJet_msoftdrop_jesPileUpMuZeroUp", "FatJet_pt_jesPileUpEnvelopeUp", "FatJet_mass_jesPileUpEnvelopeUp", "FatJet_msoftdrop_jesPileUpEnvelopeUp", "FatJet_pt_jesSubTotalPileUpUp", "FatJet_mass_jesSubTotalPileUpUp", "FatJet_msoftdrop_jesSubTotalPileUpUp", "FatJet_pt_jesSubTotalRelativeUp", "FatJet_mass_jesSubTotalRelativeUp", "FatJet_msoftdrop_jesSubTotalRelativeUp", "FatJet_pt_jesSubTotalPtUp", "FatJet_mass_jesSubTotalPtUp", "FatJet_msoftdrop_jesSubTotalPtUp", "FatJet_pt_jesSubTotalScaleUp", "FatJet_mass_jesSubTotalScaleUp", "FatJet_msoftdrop_jesSubTotalScaleUp", "FatJet_pt_jesSubTotalAbsoluteUp", "FatJet_mass_jesSubTotalAbsoluteUp", "FatJet_msoftdrop_jesSubTotalAbsoluteUp", "FatJet_pt_jesSubTotalMCUp", "FatJet_mass_jesSubTotalMCUp", "FatJet_msoftdrop_jesSubTotalMCUp", "FatJet_pt_jesTotalUp", "FatJet_mass_jesTotalUp", "FatJet_msoftdrop_jesTotalUp", "FatJet_pt_jesTotalNoFlavorUp", "FatJet_mass_jesTotalNoFlavorUp", "FatJet_msoftdrop_jesTotalNoFlavorUp", "FatJet_pt_jesTotalNoTimeUp", "FatJet_mass_jesTotalNoTimeUp", "FatJet_msoftdrop_jesTotalNoTimeUp", "FatJet_pt_jesTotalNoFlavorNoTimeUp", "FatJet_mass_jesTotalNoFlavorNoTimeUp", "FatJet_msoftdrop_jesTotalNoFlavorNoTimeUp", "FatJet_pt_jesFlavorZJetUp", "FatJet_mass_jesFlavorZJetUp", "FatJet_msoftdrop_jesFlavorZJetUp", "FatJet_pt_jesFlavorPhotonJetUp", "FatJet_mass_jesFlavorPhotonJetUp", "FatJet_msoftdrop_jesFlavorPhotonJetUp", "FatJet_pt_jesFlavorPureGluonUp", "FatJet_mass_jesFlavorPureGluonUp", "FatJet_msoftdrop_jesFlavorPureGluonUp", "FatJet_pt_jesFlavorPureQuarkUp", "FatJet_mass_jesFlavorPureQuarkUp", "FatJet_msoftdrop_jesFlavorPureQuarkUp", "FatJet_pt_jesFlavorPureCharmUp", "FatJet_mass_jesFlavorPureCharmUp", "FatJet_msoftdrop_jesFlavorPureCharmUp", "FatJet_pt_jesFlavorPureBottomUp", "FatJet_mass_jesFlavorPureBottomUp", "FatJet_msoftdrop_jesFlavorPureBottomUp", "FatJet_pt_jesTimeRunBUp", "FatJet_mass_jesTimeRunBUp", "FatJet_msoftdrop_jesTimeRunBUp", "FatJet_pt_jesTimeRunCUp", "FatJet_mass_jesTimeRunCUp", "FatJet_msoftdrop_jesTimeRunCUp", "FatJet_pt_jesTimeRunDEUp", "FatJet_mass_jesTimeRunDEUp", "FatJet_msoftdrop_jesTimeRunDEUp", "FatJet_pt_jesTimeRunFUp", "FatJet_mass_jesTimeRunFUp", "FatJet_msoftdrop_jesTimeRunFUp", "FatJet_pt_jesCorrelationGroupMPFInSituUp", "FatJet_mass_jesCorrelationGroupMPFInSituUp", "FatJet_msoftdrop_jesCorrelationGroupMPFInSituUp", "FatJet_pt_jesCorrelationGroupIntercalibrationUp", "FatJet_mass_jesCorrelationGroupIntercalibrationUp", "FatJet_msoftdrop_jesCorrelationGroupIntercalibrationUp", "FatJet_pt_jesCorrelationGroupbJESUp", "FatJet_mass_jesCorrelationGroupbJESUp", "FatJet_msoftdrop_jesCorrelationGroupbJESUp", "FatJet_pt_jesCorrelationGroupFlavorUp", "FatJet_mass_jesCorrelationGroupFlavorUp", "FatJet_msoftdrop_jesCorrelationGroupFlavorUp", "FatJet_pt_jesCorrelationGroupUncorrelatedUp", "FatJet_mass_jesCorrelationGroupUncorrelatedUp", "FatJet_msoftdrop_jesCorrelationGroupUncorrelatedUp", "FatJet_pt_jesHEMIssueUp", "FatJet_mass_jesHEMIssueUp", "FatJet_msoftdrop_jesHEMIssueUp", "FatJet_pt_jerDown", "FatJet_mass_jerDown", "FatJet_mass_jmrDown", "FatJet_mass_jmsDown", "FatJet_msoftdrop_jerDown", "FatJet_msoftdrop_tau21DDT_jerDown", "FatJet_msoftdrop_jmrDown", "FatJet_msoftdrop_jmsDown", "FatJet_msoftdrop_tau21DDT_jmrDown", "FatJet_msoftdrop_tau21DDT_jmsDown", "FatJet_pt_jesAbsoluteStatDown", "FatJet_mass_jesAbsoluteStatDown", "FatJet_msoftdrop_jesAbsoluteStatDown", "FatJet_pt_jesAbsoluteScaleDown", "FatJet_mass_jesAbsoluteScaleDown", "FatJet_msoftdrop_jesAbsoluteScaleDown", "FatJet_pt_jesAbsoluteFlavMapDown", "FatJet_mass_jesAbsoluteFlavMapDown", "FatJet_msoftdrop_jesAbsoluteFlavMapDown", "FatJet_pt_jesAbsoluteMPFBiasDown", "FatJet_mass_jesAbsoluteMPFBiasDown", "FatJet_msoftdrop_jesAbsoluteMPFBiasDown", "FatJet_pt_jesFragmentationDown", "FatJet_mass_jesFragmentationDown", "FatJet_msoftdrop_jesFragmentationDown", "FatJet_pt_jesSinglePionECALDown", "FatJet_mass_jesSinglePionECALDown", "FatJet_msoftdrop_jesSinglePionECALDown", "FatJet_pt_jesSinglePionHCALDown", "FatJet_mass_jesSinglePionHCALDown", "FatJet_msoftdrop_jesSinglePionHCALDown", "FatJet_pt_jesFlavorQCDDown", "FatJet_mass_jesFlavorQCDDown", "FatJet_msoftdrop_jesFlavorQCDDown", "FatJet_pt_jesTimePtEtaDown", "FatJet_mass_jesTimePtEtaDown", "FatJet_msoftdrop_jesTimePtEtaDown", "FatJet_pt_jesRelativeJEREC1Down", "FatJet_mass_jesRelativeJEREC1Down", "FatJet_msoftdrop_jesRelativeJEREC1Down", "FatJet_pt_jesRelativeJEREC2Down", "FatJet_mass_jesRelativeJEREC2Down", "FatJet_msoftdrop_jesRelativeJEREC2Down", "FatJet_pt_jesRelativeJERHFDown", "FatJet_mass_jesRelativeJERHFDown", "FatJet_msoftdrop_jesRelativeJERHFDown", "FatJet_pt_jesRelativePtBBDown", "FatJet_mass_jesRelativePtBBDown", "FatJet_msoftdrop_jesRelativePtBBDown", "FatJet_pt_jesRelativePtEC1Down", "FatJet_mass_jesRelativePtEC1Down", "FatJet_msoftdrop_jesRelativePtEC1Down", "FatJet_pt_jesRelativePtEC2Down", "FatJet_mass_jesRelativePtEC2Down", "FatJet_msoftdrop_jesRelativePtEC2Down", "FatJet_pt_jesRelativePtHFDown", "FatJet_mass_jesRelativePtHFDown", "FatJet_msoftdrop_jesRelativePtHFDown", "FatJet_pt_jesRelativeBalDown", "FatJet_mass_jesRelativeBalDown", "FatJet_msoftdrop_jesRelativeBalDown", "FatJet_pt_jesRelativeSampleDown", "FatJet_mass_jesRelativeSampleDown", "FatJet_msoftdrop_jesRelativeSampleDown", "FatJet_pt_jesRelativeFSRDown", "FatJet_mass_jesRelativeFSRDown", "FatJet_msoftdrop_jesRelativeFSRDown", "FatJet_pt_jesRelativeStatFSRDown", "FatJet_mass_jesRelativeStatFSRDown", "FatJet_msoftdrop_jesRelativeStatFSRDown", "FatJet_pt_jesRelativeStatECDown", "FatJet_mass_jesRelativeStatECDown", "FatJet_msoftdrop_jesRelativeStatECDown", "FatJet_pt_jesRelativeStatHFDown", "FatJet_mass_jesRelativeStatHFDown", "FatJet_msoftdrop_jesRelativeStatHFDown", "FatJet_pt_jesPileUpDataMCDown", "FatJet_mass_jesPileUpDataMCDown", "FatJet_msoftdrop_jesPileUpDataMCDown", "FatJet_pt_jesPileUpPtRefDown", "FatJet_mass_jesPileUpPtRefDown", "FatJet_msoftdrop_jesPileUpPtRefDown", "FatJet_pt_jesPileUpPtBBDown", "FatJet_mass_jesPileUpPtBBDown", "FatJet_msoftdrop_jesPileUpPtBBDown", "FatJet_pt_jesPileUpPtEC1Down", "FatJet_mass_jesPileUpPtEC1Down", "FatJet_msoftdrop_jesPileUpPtEC1Down", "FatJet_pt_jesPileUpPtEC2Down", "FatJet_mass_jesPileUpPtEC2Down", "FatJet_msoftdrop_jesPileUpPtEC2Down", "FatJet_pt_jesPileUpPtHFDown", "FatJet_mass_jesPileUpPtHFDown", "FatJet_msoftdrop_jesPileUpPtHFDown", "FatJet_pt_jesPileUpMuZeroDown", "FatJet_mass_jesPileUpMuZeroDown", "FatJet_msoftdrop_jesPileUpMuZeroDown", "FatJet_pt_jesPileUpEnvelopeDown", "FatJet_mass_jesPileUpEnvelopeDown", "FatJet_msoftdrop_jesPileUpEnvelopeDown", "FatJet_pt_jesSubTotalPileUpDown", "FatJet_mass_jesSubTotalPileUpDown", "FatJet_msoftdrop_jesSubTotalPileUpDown", "FatJet_pt_jesSubTotalRelativeDown", "FatJet_mass_jesSubTotalRelativeDown", "FatJet_msoftdrop_jesSubTotalRelativeDown", "FatJet_pt_jesSubTotalPtDown", "FatJet_mass_jesSubTotalPtDown", "FatJet_msoftdrop_jesSubTotalPtDown", "FatJet_pt_jesSubTotalScaleDown", "FatJet_mass_jesSubTotalScaleDown", "FatJet_msoftdrop_jesSubTotalScaleDown", "FatJet_pt_jesSubTotalAbsoluteDown", "FatJet_mass_jesSubTotalAbsoluteDown", "FatJet_msoftdrop_jesSubTotalAbsoluteDown", "FatJet_pt_jesSubTotalMCDown", "FatJet_mass_jesSubTotalMCDown", "FatJet_msoftdrop_jesSubTotalMCDown", "FatJet_pt_jesTotalDown", "FatJet_mass_jesTotalDown", "FatJet_msoftdrop_jesTotalDown", "FatJet_pt_jesTotalNoFlavorDown", "FatJet_mass_jesTotalNoFlavorDown", "FatJet_msoftdrop_jesTotalNoFlavorDown", "FatJet_pt_jesTotalNoTimeDown", "FatJet_mass_jesTotalNoTimeDown", "FatJet_msoftdrop_jesTotalNoTimeDown", "FatJet_pt_jesTotalNoFlavorNoTimeDown", "FatJet_mass_jesTotalNoFlavorNoTimeDown", "FatJet_msoftdrop_jesTotalNoFlavorNoTimeDown", "FatJet_pt_jesFlavorZJetDown", "FatJet_mass_jesFlavorZJetDown", "FatJet_msoftdrop_jesFlavorZJetDown", "FatJet_pt_jesFlavorPhotonJetDown", "FatJet_mass_jesFlavorPhotonJetDown", "FatJet_msoftdrop_jesFlavorPhotonJetDown", "FatJet_pt_jesFlavorPureGluonDown", "FatJet_mass_jesFlavorPureGluonDown", "FatJet_msoftdrop_jesFlavorPureGluonDown", "FatJet_pt_jesFlavorPureQuarkDown", "FatJet_mass_jesFlavorPureQuarkDown", "FatJet_msoftdrop_jesFlavorPureQuarkDown", "FatJet_pt_jesFlavorPureCharmDown", "FatJet_mass_jesFlavorPureCharmDown", "FatJet_msoftdrop_jesFlavorPureCharmDown", "FatJet_pt_jesFlavorPureBottomDown", "FatJet_mass_jesFlavorPureBottomDown", "FatJet_msoftdrop_jesFlavorPureBottomDown", "FatJet_pt_jesTimeRunBDown", "FatJet_mass_jesTimeRunBDown", "FatJet_msoftdrop_jesTimeRunBDown", "FatJet_pt_jesTimeRunCDown", "FatJet_mass_jesTimeRunCDown", "FatJet_msoftdrop_jesTimeRunCDown", "FatJet_pt_jesTimeRunDEDown", "FatJet_mass_jesTimeRunDEDown", "FatJet_msoftdrop_jesTimeRunDEDown", "FatJet_pt_jesTimeRunFDown", "FatJet_mass_jesTimeRunFDown", "FatJet_msoftdrop_jesTimeRunFDown", "FatJet_pt_jesCorrelationGroupMPFInSituDown", "FatJet_mass_jesCorrelationGroupMPFInSituDown", "FatJet_msoftdrop_jesCorrelationGroupMPFInSituDown", "FatJet_pt_jesCorrelationGroupIntercalibrationDown", "FatJet_mass_jesCorrelationGroupIntercalibrationDown", "FatJet_msoftdrop_jesCorrelationGroupIntercalibrationDown", "FatJet_pt_jesCorrelationGroupbJESDown", "FatJet_mass_jesCorrelationGroupbJESDown", "FatJet_msoftdrop_jesCorrelationGroupbJESDown", "FatJet_pt_jesCorrelationGroupFlavorDown", "FatJet_mass_jesCorrelationGroupFlavorDown", "FatJet_msoftdrop_jesCorrelationGroupFlavorDown", "FatJet_pt_jesCorrelationGroupUncorrelatedDown", "FatJet_mass_jesCorrelationGroupUncorrelatedDown", "FatJet_msoftdrop_jesCorrelationGroupUncorrelatedDown", "FatJet_pt_jesHEMIssueDown", "FatJet_mass_jesHEMIssueDown", "FatJet_msoftdrop_jesHEMIssueDown", "Muon_corrected_pt", "Muon_correctedUp_pt", "Muon_correctedDown_pt", "HT_pt", "MHT_pt", "MHT_phi", "Jet_mhtCleaning" ]